# Module 2 - Core Prompting Techniques

| **Aspect** | **Details** |
|-------------|-------------|
| **Goal** | Master 8 core prompt engineering tactics: role prompting, structured inputs, few-shot examples, chain-of-thought reasoning, reference citations, prompt chaining, LLM-as-judge, and inner monologue to build professional-grade AI workflows |
| **Time** | ~90-120 minutes (1.5-2 hours) |
| **Prerequisites** | Python 3.8+, IDE with notebook support, API access (GitHub Copilot, CircuIT, or OpenAI) |
| **Setup Required** | Clone the repository and follow [Quick Setup](../README.md) before running this notebook |

---

## 🚀 Ready to Start?

<div style="margin-top:16px; color:#991b1b; padding:12px; background:#fee2e2; border-radius:6px; border-left:4px solid #ef4444;">
<strong>⚠️ Important:</strong> <br><br>
This module requires fresh setup. Even if you completed Module 1, run the setup cells below to ensure everything works correctly.<br>
</div>

## 🔧 Setup: Environment Configuration

### Step 1: Install Required Dependencies



Let's start by installing the packages we need for this tutorial.

Run the cell below. You should see a success message when installation completes:

In [ ]:
# Install required packages for Module 2
import subprocess
import sys

def install_requirements():
    try:
        # Install from requirements.txt
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"])
        print("✅ SUCCESS! Module 2 dependencies installed successfully.")
        print("📦 Ready for: openai, anthropic, python-dotenv, requests")
    except subprocess.CalledProcessError as e:
        print(f"❌ Installation failed: {e}")
        print("💡 Try running: pip install openai anthropic python-dotenv requests")

install_requirements()


### Step 2: Connect to AI Model

<div style="margin-top:16px; color:#78350f; padding:12px; background:#fef3c7; border-radius:6px; border-left:4px solid #f59e0b;">
<strong>💡 Note:</strong> <br><br>
The code below runs on your local machine and connects to AI services over the internet.
</div>

Choose your preferred option:

- **Option A: GitHub Copilot API (local proxy)** ⭐ **Recommended**: 
  - Supports both **Claude** and **OpenAI** models
  - No API keys needed - uses your GitHub Copilot subscription
  - Follow [GitHub-Copilot-2-API/README.md](../../GitHub-Copilot-2-API/README.md) to authenticate and start the local server
  - Run the setup cell below and **edit your preferred provider** (`"openai"` or `"claude"`) by setting the `PROVIDER` variable
  - Available models:
    - **OpenAI**: gpt-4o, gpt-4, gpt-3.5-turbo, o3-mini, o4-mini
    - **Claude**: claude-3.5-sonnet, claude-3.7-sonnet, claude-sonnet-4

- **Option B: OpenAI API**: If you have OpenAI API access, uncomment and run the **Option B** cell below.

- **Option C: CircuIT APIs (Azure OpenAI)**: If you have CircuIT API access, uncomment and run the **Option C** cell below.


In [ ]:
# Option A: GitHub Copilot API setup (Recommended)
import openai
import anthropic
import os

# ============================================
# 🎯 CHOOSE YOUR AI MODEL PROVIDER
# ============================================
# Set your preference: "openai" or "claude"
PROVIDER = "claude"  # Change to "claude" to use Claude models

# ============================================
# 📋 Available Models by Provider
# ============================================
# OpenAI Models (via GitHub Copilot):
#   - gpt-4o (recommended, supports vision)
#   - gpt-4
#   - gpt-3.5-turbo
#   - o3-mini, o4-mini
#
# Claude Models (via GitHub Copilot):
#   - claude-3.5-sonnet (recommended, supports vision)
#   - claude-3.7-sonnet (supports vision)
#   - claude-sonnet-4 (supports vision)
# ============================================

# Configure clients for both providers
openai_client = openai.OpenAI(
    base_url="http://localhost:7711/v1",
    api_key="dummy-key"
)

claude_client = anthropic.Anthropic(
    api_key="dummy-key",
    base_url="http://localhost:7711"
)

# Set default models for each provider
OPENAI_DEFAULT_MODEL = "gpt-4o"
CLAUDE_DEFAULT_MODEL = "claude-3.5-sonnet"


def _extract_text_from_blocks(blocks):
    """Extract text content from response blocks returned by the API."""
    parts = []
    for block in blocks:
        text_val = getattr(block, "text", None)
        if isinstance(text_val, str):
            parts.append(text_val)
        elif isinstance(block, dict):
            t = block.get("text")
            if isinstance(t, str):
                parts.append(t)
    return "\n".join(parts)


def get_openai_completion(messages, model=None, temperature=0.0):
    """Get completion from OpenAI models via GitHub Copilot."""
    if model is None:
        model = OPENAI_DEFAULT_MODEL
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ Error: {e}\n💡 Make sure GitHub Copilot proxy is running on port 7711"


def get_claude_completion(messages, model=None, temperature=0.0):
    """Get completion from Claude models via GitHub Copilot."""
    if model is None:
        model = CLAUDE_DEFAULT_MODEL
    try:
        response = claude_client.messages.create(
            model=model,
            max_tokens=8192,
            messages=messages,
            temperature=temperature
        )
        return _extract_text_from_blocks(getattr(response, "content", []))
    except Exception as e:
        return f"❌ Error: {e}\n💡 Make sure GitHub Copilot proxy is running on port 7711"


def get_chat_completion(messages, model=None, temperature=0.7):
    """
    Generic function to get chat completion from any provider.
    Routes to the appropriate provider-specific function based on PROVIDER setting.
    """
    if PROVIDER.lower() == "claude":
        return get_claude_completion(messages, model, temperature)
    else:  # Default to OpenAI
        return get_openai_completion(messages, model, temperature)


def get_default_model():
    """Get the default model for the current provider."""
    if PROVIDER.lower() == "claude":
        return CLAUDE_DEFAULT_MODEL
    else:
        return OPENAI_DEFAULT_MODEL


# ============================================
# 🧪 TEST CONNECTION
# ============================================
print("🔄 Testing connection to GitHub Copilot proxy...")
test_result = get_chat_completion([
    {"role": "user", "content": "test"}
])

if test_result and "Error" in test_result:
    print("\n" + "="*60)
    print("❌ CONNECTION FAILED!")
    print("="*60)
    print(f"Provider: {PROVIDER.upper()}")
    print(f"Expected endpoint: http://localhost:7711")
    print("\n⚠️  The GitHub Copilot proxy is NOT running!")
    print("\n📋 To fix this:")
    print("   1. Open a new terminal")
    print("   2. Navigate to your copilot-api directory")
    print("   3. Run: uv run copilot2api start")
    print("   4. Wait for the server to start (you should see 'Server initialized')")
    print("   5. Come back and rerun this cell")
    print("\n💡 Need setup help? See: GitHub-Copilot-2-API/README.md")
    print("="*70)
else:
    print("\n" + "="*60)
    print("✅ CONNECTION SUCCESSFUL!")
    print("="*60)
    print(f"🤖 Provider: {PROVIDER.upper()}")
    print(f"📦 Default Model: {get_default_model()}")
    print(f"🔗 Endpoint: http://localhost:7711")
    print(f"\n💡 To switch providers, change PROVIDER to '{'claude' if PROVIDER.lower() == 'openai' else 'openai'}' and rerun this cell")
    print("="*70)


### Option B: Direct OpenAI API

**Setup:** Add your API key to `.env` file, then uncomment and run:

> 💡 **Note:** This option requires a paid OpenAI API account. If you're using GitHub Copilot, stick with Option A above.


In [ ]:
# # Option B: Direct OpenAI API setup
# import openai
# import os
# from dotenv import load_dotenv

# load_dotenv()

# client = openai.OpenAI(
#     api_key=os.getenv("OPENAI_API_KEY")  # Set this in your .env file
# )

# def get_chat_completion(messages, model="gpt-4o", temperature=0.7):
#     """Get a chat completion from OpenAI."""
#     try:
#         response = client.chat.completions.create(
#             model=model,
#             messages=messages,
#             temperature=temperature
#         )
#         return response.choices[0].message.content
#     except Exception as e:
#         return f"❌ Error: {e}"

# print("✅ OpenAI API configured successfully!")
# print("🤖 Using OpenAI's official API")


### Option C: CircuIT APIs (Azure OpenAI)

**Setup:** Configure environment variables (`CISCO_CLIENT_ID`, `CISCO_CLIENT_SECRET`, `CISCO_OPENAI_APP_KEY`) in `.env` file.

Get values from: https://ai-chat.cisco.com/bridgeit-platform/api/home

Then uncomment and run:

> 💡 **Note:** This option is for Cisco employees with CircuIT API access.


In [ ]:
# # Option C: CircuIT APIs (Azure OpenAI) setup
# import openai
# import traceback
# import requests
# import base64
# import os
# from dotenv import load_dotenv
# from openai import AzureOpenAI

# # Load environment variables
# load_dotenv()

# # Open AI version to use
# openai.api_type = "azure"
# openai.api_version = "2024-12-01-preview"

# # Get API_KEY wrapped in token - using environment variables
# client_id = os.getenv("CISCO_CLIENT_ID")
# client_secret = os.getenv("CISCO_CLIENT_SECRET")

# url = "https://id.cisco.com/oauth2/default/v1/token"

# payload = "grant_type=client_credentials"
# value = base64.b64encode(f"{client_id}:{client_secret}".encode("utf-8")).decode("utf-8")
# headers = {
#     "Accept": "*/*",
#     "Content-Type": "application/x-www-form-urlencoded",
#     "Authorization": f"Basic {value}",
# }

# token_response = requests.request("POST", url, headers=headers, data=payload)
# print(token_response.text)
# token_data = token_response.json()

# client = AzureOpenAI(
#     azure_endpoint="https://chat-ai.cisco.com",
#     api_key=token_data.get("access_token"),
#     api_version="2024-12-01-preview",
# )

# app_key = os.getenv("CISCO_OPENAI_APP_KEY")

# def get_chat_completion(messages, model="gpt-4o", temperature=0.7):
#     """Get a chat completion from CircuIT APIs."""
#     try:
#         response = client.chat.completions.create(
#             model=model,
#             messages=messages,
#             temperature=temperature,
#             user=f'{{"appkey": "{app_key}"}}',
#         )
#         return response.choices[0].message.content
#     except Exception as e:
#         return f"❌ Error: {e}"

# print("✅ CircuIT APIs configured successfully!")
# print("🤖 Using Azure OpenAI via CircuIT")


### Step 3: Test Connection

Let's test that everything is working before we begin:

<div style="margin-top:16px; color:#78350f; padding:12px; background:#fef3c7; border-radius:6px; border-left:4px solid #f59e0b;">
<strong>💡 Tip:</strong> If you see long AI responses and the output shows "Output is truncated. View as a scrollable element" - <strong>click that link</strong> to see the full response in a scrollable view!
</div>


In [ ]:
# Quick setup verification
test_messages = [
    {
        "role": "system",
        "content": "You are a prompt engineering instructor. Respond with: 'Module 2 setup verified! Ready to learn core techniques.'"
    },
    {
        "role": "user",
        "content": "Test Module 2 setup"
    }
]

response = get_chat_completion(test_messages)
print("🧪 Setup Test:")
print(response)

if response and ("verified" in response.lower() or "ready" in response.lower()):
    print("\n🎉 Perfect! Module 2 environment is ready!")
else:
    print("\n⚠️  Setup test complete. Let's continue with the tutorial!")


---

## 🎯 Core Prompt Engineering Techniques

### Introduction: The Art of Prompt Engineering

#### 🚀 Ready to Transform Your AI Interactions?

You've successfully set up your environment and tested the connection. Now comes the exciting part - **learning the tactical secrets** that separate amateur prompt writers from AI power users.

Think of what you've accomplished so far as **laying the foundation** of a house. Now we're about to build the **architectural masterpiece** that will revolutionize how you work with AI assistants.


#### 👨‍🏫 What You're About to Master

Before diving into advanced techniques, there's one fundamental skill you must master: **writing clear instructions**. This is the foundation upon which all other tactics are built. Just as a house needs a solid foundation before adding walls and a roof, effective prompting requires clarity before applying sophisticated techniques.

In the sections that follow, you'll discover **eight core tactics** that professional developers use to get consistently excellent results from AI:

<div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 16px; margin: 20px 0;">

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>🎭 Role Prompting</strong><br>
<em>Transform AI into specialized experts</em>
</div>

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>📋 Structured Inputs</strong><br>
<em>Organize complex scenarios with precision</em>
</div>

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>📚 Few-Shot Examples</strong><br>
<em>Teach AI your preferred style</em>
</div>

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>⛓️‍💥 Chain-of-Thought</strong><br>
<em>Guide AI through systematic reasoning</em>
</div>

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>📖 Reference Citations</strong><br>
<em>Answer with citations from reference text</em>
</div>

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>🔗 Prompt Chaining</strong><br>
<em>Break complex tasks into sequential steps</em>
</div>

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>⚖️ LLM-as-Judge</strong><br>
<em>Use AI to evaluate and improve outputs</em>
</div>

<div style="background: #f8fafc; border: 2px solid #e2e8f0; border-radius: 8px; padding: 16px; text-align: center; color: #000000;">
<strong>🤫 Inner Monologue</strong><br>
<em>Hide reasoning, show only final results</em>
</div>

</div>

<div style="margin-top:16px; color:#15803d; padding:12px; background:#dcfce7; border-radius:6px; border-left:4px solid #22c55e;">
<strong>💡 Pro Tip:</strong> <br><br>
This module covers 8 powerful tactics over 90-120 minutes. <strong>Take short breaks</strong> between tactics to reflect on how you can apply each technique to your day-to-day work. <strong>Make notes</strong> as you progress—jot down specific use cases from your projects where each tactic could be valuable. This active reflection will help you retain the techniques and integrate them into your workflow faster!
</div>

---

### 📍 How to Use Break Points

<div style="background:#f0f9ff; border-left:4px solid #3b82f6; padding:16px; border-radius:6px; margin:20px 0; color:#000000;">
<strong style="color:#1e40af;">💡 Taking Breaks? We've Got You Covered!</strong><br><br>

This module is designed for 90-120 minutes of focused learning. To help you manage your time effectively, we've added **4 strategic break points** throughout.

**Why One Notebook?** This tutorial uses a single notebook because the setup cells (API connections, helper functions) need to remain active throughout all tactics. Splitting into multiple notebooks would require re-running setup in each file, creating a fragmented learning experience. The break points below let you pause and resume seamlessly within one cohesive environment.

<table style="width:100%; margin:10px 0; border-collapse: collapse;">
  <tr style="background:#dbeafe;">
    <th style="padding:8px; text-align:left; border:1px solid #93c5fd;">Break Point</th>
    <th style="padding:8px; text-align:left; border:1px solid #93c5fd;">Location</th>
    <th style="padding:8px; text-align:left; border:1px solid #93c5fd;">Time Elapsed</th>
    <th style="padding:8px; text-align:left; border:1px solid #93c5fd;">Bookmark Text</th>
  </tr>
  <tr>
    <td style="padding:8px; border:1px solid #93c5fd;">☕ Break #1</td>
    <td style="padding:8px; border:1px solid #93c5fd;">After Tactic 2</td>
    <td style="padding:8px; border:1px solid #93c5fd;">~30 min</td>
    <td style="padding:8px; border:1px solid #93c5fd;">"Tactic 3: Few-Shot Examples"</td>
  </tr>
  <tr style="background:#eff6ff;">
    <td style="padding:8px; border:1px solid #93c5fd;">🍵 Break #2</td>
    <td style="padding:8px; border:1px solid #93c5fd;">After Tactic 4</td>
    <td style="padding:8px; border:1px solid #93c5fd;">~60 min</td>
    <td style="padding:8px; border:1px solid #93c5fd;">"Tactic 5: Reference Citations"</td>
  </tr>
  <tr>
    <td style="padding:8px; border:1px solid #93c5fd;">🧃 Break #3</td>
    <td style="padding:8px; border:1px solid #93c5fd;">After Tactic 6</td>
    <td style="padding:8px; border:1px solid #93c5fd;">~90 min</td>
    <td style="padding:8px; border:1px solid #93c5fd;">"Tactic 7: LLM-as-Judge"</td>
  </tr>
  <tr style="background:#eff6ff;">
    <td style="padding:8px; border:1px solid #93c5fd;">🎯 Break #4</td>
    <td style="padding:8px; border:1px solid #93c5fd;">Before Practice</td>
    <td style="padding:8px; border:1px solid #93c5fd;">~100 min</td>
    <td style="padding:8px; border:1px solid #93c5fd;">"Hands-On Practice - Activity 2.1"</td>
  </tr>
</table>

**How to Resume Your Session:**
1. Scroll down to find the colorful break point card you last saw
2. Look for the **"📌 BOOKMARK TO RESUME"** section
3. Use `Ctrl+F` (or `Cmd+F` on Mac) to search for the bookmark text
4. You'll jump right to where you left off!

**Pro Tip:** Each break point card shows:
- ✅ What you've completed
- ⏭️ What's coming next
- ⏱️ Estimated time for the next section

Feel free to work at your own pace—these are suggestions, not requirements! 🚀
</div>

---

### 🎬 Tactic 0: Write Clear Instructions

Now that you understand how clear instructions form the foundation for all other tactics, let's explore what makes instructions truly effective.

**Core Principle:** When interacting with AI models, think of them as brilliant but very new employees who need explicit instructions. The more precisely you explain what you want—including context, specific requirements, and sequential steps—the better the AI's response will be.

Show your prompt to a colleague with minimal context on the task. If they're confused, the AI will likely be too. This becomes crucial when asking for code refactoring, where you need to specify coding standards, performance requirements, and constraints to get production-ready results.

*Reference: [Claude Documentation - Be Clear and Direct](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/be-clear-and-direct)*

#### Example: Vague vs. Specific Instructions

**Why This Works:** Specific instructions eliminate ambiguity and guide the model toward your exact requirements.

Let's compare a generic approach with a specific one:


In [ ]:
# Vague request - typical beginner mistake
messages = [
    {"role": "user", "content": "Help me choose a programming language for my project"}
]

response = get_chat_completion(messages)

print("VAGUE REQUEST RESULT:")
print(response)
print("\n" + "="*50 + "\n")

In [ ]:
# Specific request - much better results
messages = [
    {
        "role": "user",
        "content": "I need to choose a programming language for building a real-time chat application that will handle 10,000 concurrent users, needs to integrate with a PostgreSQL database, and must be deployable on AWS. The team has 3 years of experience with web development. Provide the top 3 language recommendations with pros and cons for each.",
    }
]

response = get_chat_completion(messages)

print("SPECIFIC REQUEST RESULT:")
print(response)
print("\n" + "="*50 + "\n")

Another way to achieve specificity is using the **system prompt**. The system prompt is a special message type in the conversation structure (alongside "user" and "assistant" messages) that sets overarching instructions, context, and behavioral guidelines for the AI before the user asks their question. Think of it as setting the "rules of engagement" for the entire conversation. It's particularly useful when you want to keep the user request clean while providing detailed instructions about response format, tone, constraints, and expertise level that should apply to all responses.

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a senior technical architect. Provide concise, actionable recommendations in bullet format. Focus only on the most critical factors for the decision. No lengthy explanations.",
    },
    {
        "role": "user",
        "content": "Help me choose between microservices and monolithic architecture for a startup with 5 developers building a fintech application",
    },
]

response = get_chat_completion(messages)

print("SYSTEM PROMPT RESULT:")
print(response)
print("\n" + "="*50 + "\n")

---

### 🎯 Try It Yourself: Clear Instructions

**Common Misconception:** AI can understand vague requests and "read your mind" about what you need.

**The Reality:** Specific instructions dramatically improve output quality.

**Your Task:** Below is a vague prompt. Your job is to rewrite it with clear, specific instructions that include:
- Context about the project
- Specific requirements
- Constraints or preferences
- Expected output format

Run both versions and compare the results!

In [ ]:
# ❌ BAD: Vague prompt
bad_messages = [
    {"role": "user", "content": "Help me optimize my database"}
]

bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("VAGUE PROMPT RESULT:")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Rewrite with specific instructions
# TODO: Uncomment and complete this section
# good_messages = [
#     {
#         "role": "user",
#         "content": """
# I have a PostgreSQL database for an e-commerce site with 1 million products.
# The product search query is taking 5+ seconds. Here's the current query:
# 
# SELECT * FROM products WHERE name LIKE '%search_term%' OR description LIKE '%search_term%'
# 
# Requirements:
# - Reduce query time to under 1 second
# - Must support partial text matching
# - Can't change the database schema (production constraint)
# - Provide top 3 optimization techniques
# 
# Format response as:
# 1. Technique name
# 2. Implementation steps
# 3. Expected performance improvement
# """
#     }
# ]
# 
# good_response = get_chat_completion(good_messages)
# print("=" * 70)
# print("SPECIFIC PROMPT RESULT:")
# print("=" * 70)
# print(good_response)

#### Understanding Message Structure

<div style="margin-top:16px; color:#1e40af; padding:12px; background:#dbeafe; border-radius:6px; border-left:4px solid #3b82f6;">
<strong>📝 Note:</strong> Throughout this tutorial, we structure prompts as JSON with three message types:
<ul style="margin: 8px 0;">
<li><strong>system:</strong> Sets overall instructions and behavior for the AI</li>
<li><strong>user:</strong> Contains the actual question or task</li>
<li><strong>assistant:</strong> Used for few-shot examples (you'll see this in Tactic 3)</li>
</ul>
</div>

---

### 🎭 Tactic 1: Role Prompting

**Transform AI into specialized domain experts**

You can use role prompting as a way to get AI models to emulate certain styles in writing, speak in a certain voice, or guide the complexity of their answers. Role prompting can also make AI models better at performing math or logic tasks.

Using the `system` parameter for role prompting is the most powerful way to transform any LLM from a general assistant into your virtual domain expert. The right role enhances accuracy in complex scenarios, tailors the communication tone, and improves focus by keeping the LLM within the bounds of your task's specific requirements.

In coding scenarios, role prompting helps with tasks like specific refactoring requirements (e.g., "Extract this into separate classes following SOLID principles"), detailed code review criteria (e.g., "Focus on security vulnerabilities and performance bottlenecks"), and precise testing specifications (e.g., "Generate unit tests with 90% coverage including edge cases").

*Reference: [Claude Documentation - System Prompts](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/system-prompts)*

In [ ]:
# Instead of asking for a generic response, adopt a specific persona
messages = [
    {
        "role": "system",
        "content": "You are a code reviewer. Analyze the provided code and give exactly 3 specific feedback points: 1 about code structure, 1 about naming conventions, and 1 about potential improvements. Format each point as a bullet with the category in brackets.",
    },
    {
        "role": "user",
        "content": "def calc(x, y): return x + y if x > 0 and y > 0 else 0",
    },
]
response = get_chat_completion(messages)

print("CODE REVIEWER PERSONA RESULT:")
print(response)
print("\n" + "="*50 + "\n")

#### Software Engineering Personas

The below cells show how different engineering personas provide specialized expertise for code reviews.

In [ ]:
# Security Engineer Persona
security_messages = [
    {
        "role": "system", 
        "content": "You are a security engineer. Review code for security vulnerabilities and provide specific recommendations."
    },
    {
        "role": "user",
        "content": """Review this login function:
        
def login(username, password):
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    result = database.execute(query)
    return result"""
    }
]

security_response = get_chat_completion(security_messages)
print("🔒 SECURITY ENGINEER ANALYSIS:")
print(security_response)
print("\n" + "="*50 + "\n")


---

### 🎯 Try It Yourself: Role Prompting

**Common Misconception:** Generic code review requests produce the same quality results regardless of expertise focus.

**The Reality:** Different personas catch different issues. A security engineer spots vulnerabilities a performance engineer might miss.

**Your Task:** Below is a generic code review prompt. Fix it by:
1. Adding an appropriate role in the `system` message
2. Choosing between: Security Engineer, Performance Engineer, or QA Engineer
3. Specifying what that persona should focus on

Compare the generic review with your specialized review!

In [ ]:
# Code to review
code_to_review = """
def get_user_data(user_id):
    query = f"SELECT * FROM users WHERE id = {user_id}"
    results = db.execute(query).fetchall()
    return results
"""

# ❌ BAD: Generic review (no specific expertise)
bad_messages = [
    {
        "role": "user",
        "content": f"Review this code:\n\n{code_to_review}"
    }
]

bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("GENERIC REVIEW (No Role):")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Add a role and specific focus
# TODO: Uncomment and complete this section
# Choose a role: Security Engineer, Performance Engineer, or QA Engineer
# good_messages = [
#     {
#         "role": "system",
#         "content": "You are a Security Engineer. Review code specifically for security vulnerabilities including SQL injection, authentication issues, and data exposure risks. Provide severity ratings."
#     },
#     {
#         "role": "user",
#         "content": f"Review this code:\n\n{code_to_review}"
#     }
# ]
# 
# good_response = get_chat_completion(good_messages)
# print("=" * 70)
# print("SECURITY ENGINEER REVIEW (With Role):")
# print("=" * 70)
# print(good_response)
# 
# print("\n💡 Notice the difference? The security engineer immediately spots SQL injection!"

---
### 📋 Tactic 2: Structured Inputs

**Organize complex scenarios with delimiters**

When your prompts involve multiple components like context, instructions, and examples, it can be a game-changer to use delimiters that clearly separate these parts. Delimiters help AI models parse your prompts more accurately, leading to higher-quality outputs.

<div style="margin:16px 0; padding:12px; background:#dbeafe; border-radius:6px; border-left:4px solid #3b82f6; color:#1f2937;">
<style>
code {
  font-family: Consolas,"courier new";
  color:rgb(238, 13, 13);
  background-color: #f1f1f1;
  padding: 2px;
  font-size: 110%;
}
</style>
<strong style="color:#1e40af;">📌 Building on Previous Tactics:</strong><br><br>
While <em>Tactics 0</em> and <em>1</em> used basic structure (separating system/user messages and organizing information in JSON format), Tactic 2 takes structure further by using explicit delimiters <em>within</em> your prompt content to organize complex, multi-part information.
<br><br>
<strong>Note:</strong> JSON format is required to pass messages to the <code>get_chat_completion()</code> function - that's the API's message structure. Tactic 2 adds delimiters <em>inside</em> the message content itself for better organization.
</div>

**Understanding Delimiters**

* Simple Delimiters
  - Use `###` or `#` to create visual section headers within your prompts
  - These act like markdown headers, making sections distinct and easy to identify
  - Example: `### CODE ###` followed by code, then `### REQUIREMENTS ###` followed by requirements
  - The LLM recognizes these as visual separators that indicate different content sections

* XML Delimiters
  - More powerful and precise than simple delimiters
  - LLMs are extensively trained on XML/HTML during their training, making them particularly good at parsing tagged content
  - You can create your own tag names that are descriptive (e.g., `<user_input>`, `<system_logs>`, `<test_cases>`)
  - Tag names should be meaningful and describe their content - while you have flexibility, clear names improve results
  - Common effective tags: `<instructions>`, `<example>`, `<context>`, `<requirements>`, `<code>`, `<output>`
  - LLMs have seen standard HTML/XML tags during training (like `<document>`, `<source>`, `<content>`), which is why the Claude documentation recommends certain tags

**Why This Works:**
- **Clarity:** Clearly separate different parts of your prompt and ensure your prompt is well structured
- **Accuracy:** Reduce errors caused by AI models misinterpreting parts of your prompt  
- **Flexibility:** Easily find, add, remove, or modify parts of your prompt without rewriting everything
- **Parseability:** Having the AI use delimiters in its output makes it easier to extract specific parts of its response

In multi-file refactoring, separating different files being modified becomes essential using delimiters like `<original_file>` and `<refactored_file>`. You can distinguish between `<requirements>` and `<existing_code>`, organize `<test_cases>`, `<edge_cases>`, and `<error_cases>`, and structure pull request reviews with `<pr_description>`, `<code_changes>`, and `<test_changes>`.

Let's start with a simple example showing how delimiters clarify different sections of your prompt by using `###` as delimiters:

In [ ]:
# Using delimiters to refactor code
function_code = "def process_data(items): return [x.upper() for x in items if len(x) > 3]"
requirements = "Follow PEP 8 style guide, add type hints, improve readability"

delimiter_messages = [
    {
        "role": "system",
        "content": "You are a Python code reviewer. Provide only the refactored code without explanations."
    },
    {
        "role": "user",
        "content": f"""Refactor this function based on the requirements:

### CODE ###
{function_code}
###

### REQUIREMENTS ###
{requirements}
###

Return only the improved function code."""
    }
]

delimiter_response = get_chat_completion(delimiter_messages)
print("🔧 REFACTORED CODE:")
print(delimiter_response)
print("\n" + "="*70 + "\n")

#### Multi-File Scenarios with XML Delimiters

One of the most powerful techniques for complex software development tasks is using XML tags and delimiters to structure your prompts. This approach dramatically improves AI accuracy and reduces misinterpretation.

**Key Benefits:**
- **Clarity**: Clearly separate different parts of your prompt (instructions, context, examples)
- **Accuracy**: Reduce errors caused by AI misinterpreting parts of your prompt
- **Flexibility**: Easily modify specific sections without rewriting everything
- **Parseability**: Structure AI outputs for easier post-processing

**Best Practices:**
- Use tags like `<instructions>`, `<example>`, and `<formatting>` to clearly separate different parts
- Be consistent with tag names throughout your prompts
- Nest tags hierarchically: `<outer><inner></inner></outer>` for structured content
- Choose meaningful tag names that describe their content

**Reference**: Learn more about XML tagging best practices in the [Claude Documentation on XML Tags](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags).

In coding scenarios, delimiters become essential for:

- **Multi-file refactoring** - Separate different files being modified: `<original_file>`, `<refactored_file>`
- **Code vs. requirements** - Distinguish between `<requirements>` and `<existing_code>`
- **Test scenarios** - Organize `<test_cases>`, `<edge_cases>`, `<error_cases>`
- **Pull request reviews** - Structure `<pr_description>`, `<code_changes>`, `<test_changes>`

The below cell demonstrates multi-file refactoring using XML delimiters to organize complex codebases.

In [ ]:
# Multi-file analysis with XML delimiters
multifile_messages = [
    {
        "role": "system",
        "content": "You are a software architect. Analyze the provided files and identify architectural concerns."
    },
    {
        "role": "user",
        "content": """
<user_model>
class User:
    def __init__(self, email, password):
        self.email = email
        self.password = password
    
    def save(self):
        # Save to database
        pass
</user_model>

<user_controller>
from flask import Flask, request
app = Flask(__name__)

@app.route('/register', methods=['POST'])
def register():
    email = request.form['email']
    password = request.form['password']
    user = User(email, password)
    user.save()
    return "User registered"
</user_controller>

<requirements>
- Follow separation of concerns
- Add input validation
- Implement proper error handling
- Use dependency injection
</requirements>

Provide architectural recommendations for improving this code structure.
"""
    }
]

multifile_response = get_chat_completion(multifile_messages)
print("🏗️ ARCHITECTURAL ANALYSIS:")
print(multifile_response)
print("\n" + "="*70 + "\n")

---

### 🎯 Try It Yourself: Structured Inputs

**Common Misconception:** AI can parse messy, unorganized prompts just as well as structured ones.

**The Reality:** Delimiters dramatically reduce misinterpretation and improve accuracy, especially with complex multi-part inputs.

**Your Task:** Below is a messy prompt with requirements, code, and context all jumbled together. Reorganize it using XML tags:
- `<requirements>` for what needs to be done
- `<current_code>` for the existing implementation
- `<context>` for background information

Run both versions and see how structure improves the analysis!

In [ ]:
# ❌ BAD: Messy, unstructured prompt
bad_messages = [
    {
        "role": "user",
        "content": """
I need to refactor this function: def send_email(to, subject, body): smtp.send(to, subject, body)

The requirements are: add error handling, implement retry logic with exponential backoff, add logging, and validate email format. This is for a high-traffic notification service that sends 10k emails per hour. The current implementation fails silently when SMTP server is down and doesn't validate email addresses. We need 99.9% delivery rate.

Please refactor this code following best practices.
"""
    }
]

bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("MESSY PROMPT RESULT:")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Restructure with XML tags
# TODO: Uncomment and complete this section
# good_messages = [
#     {
#         "role": "user",
#         "content": """
# <current_code>
# def send_email(to, subject, body):
#     smtp.send(to, subject, body)
# </current_code>
# 
# <context>
# This is for a high-traffic notification service that sends 10k emails per hour.
# Current implementation fails silently when SMTP server is down.
# Email addresses are not validated.
# We need 99.9% delivery rate.
# </context>
# 
# <requirements>
# 1. Add error handling
# 2. Implement retry logic with exponential backoff
# 3. Add logging for monitoring
# 4. Validate email format before sending
# 5. Follow Python best practices
# </requirements>
# 
# Please refactor this code addressing all requirements.
# """
#     }
# ]
# 
# good_response = get_chat_completion(good_messages)
# print("=" * 70)
# print("STRUCTURED PROMPT RESULT:")
# print("=" * 70)
# print(good_response)
# 
# print("\n💡 Notice how the structured version produces more organized, complete refactoring!")

---

### 📚 Tactic 3: Few-Shot Examples

**Teach AI your preferred styles and standards through carefully crafted examples**

Providing examples is your secret weapon for consistent, accurate outputs. This technique (called "few-shot" or "multishot" prompting) is especially effective for structured outputs and specific formats.

**What "Styles and Standards" Means:**
Coding conventions • Documentation formats • Error message patterns • Commit message styles • Test case patterns • API response structures

**Few-Shot Terminology:**

The number of "shots" = how many examples you provide:
- **Zero-shot:** No examples (instructions only)
- **One-shot:** Single example
- **Few-shot:** 2-5 examples (sweet spot for most tasks)
- **N-shot:** Many examples for complex patterns

**Why Examples Work:**

Showing AI how you want it to behave (or *not* behave) is powerful for:
1. **Right answer:** Clarifies ambiguous requirements
2. **Right format:** Demonstrates exact structure and style
3. **Pattern learning:** AI infers rules you'd struggle to describe
4. **Less confusion:** Eliminates guesswork

**Best Practices:**
- Mirror your actual use case
- Include 3-5 diverse examples for best results
- Wrap in `<example>` tags (or `<examples>` for multiple)
- Cover edge cases without creating unintended patterns
- More examples = better performance for complex tasks

*Reference: [Claude Documentation - Multishot Prompting](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/multishot-prompting)*

Let's teach the AI to extract structured data from logs in a consistent format:

In [ ]:
# Few-shot examples for consistent log parsing
few_shot_messages = [
    {"role": "system", "content": "Extract service names and error types from log entries following the examples provided."},
    
    # Example 1
    {"role": "user", "content": 'Extract from: "[ERROR] payment-service: Database connection pool exhausted"'},
    {"role": "assistant", "content": "Service: payment-service, Error: connection_pool"},
    
    # Example 2  
    {"role": "user", "content": 'Extract from: "[WARN] user-auth: Rate limit exceeded for API endpoint"'},
    {"role": "assistant", "content": "Service: user-auth, Error: rate_limit"},
    
    # Example 3
    {"role": "user", "content": 'Extract from: "[ERROR] notification-hub: Message queue timeout after 30s"'},
    {"role": "assistant", "content": "Service: notification-hub, Error: timeout"},
    
    # New log entry following the established pattern
    {"role": "user", "content": 'Extract from: "[ERROR] inventory-manager: Cache invalidation failed during peak load"'}
]

few_shot_response = get_chat_completion(few_shot_messages)
print("📚 CONSISTENT LOG PARSING RESPONSE:")
print(few_shot_response)
print("\n" + "="*70 + "\n")

🎯 **Perfect!** Notice how the AI learned the exact format and style from the examples and applied it consistently.

**Why This Works - System Messages + Examples:**

This example demonstrates how **few-shot learning differs from role prompting**:

- **Role Prompting (Tactic 1):** Assigns AI a *persona* or *expertise* (e.g., "You are a security engineer") → Focuses on domain knowledge and perspective
- **Few-Shot Examples (Tactic 3):** Teaches AI a *pattern* or *style* through concrete examples → Focuses on format, structure, and consistency

**The Mechanism:**
- The `system` message "Answer in a consistent style using the examples provided" primes the AI to look for patterns
- The `user`/`assistant` pairs demonstrate the desired input-output relationship
- The AI learns: "concise format," "specific structure," "technical accuracy" from seeing 2-3 examples
- When given a new question, it applies the learned pattern automatically

**Why Both Matter:**
- Use role prompting when you need **specialized knowledge** (security vulnerabilities, performance optimization)
- Use few-shot examples when you need **consistent formatting** (commit messages, error patterns, documentation style)
- Combine both for powerful results: "You are a senior engineer" (role) + 3 examples of your team's code review format (pattern)


---

### 🎯 Try It Yourself: Few-Shot Examples

**Common Misconception:** AI automatically knows your preferred categorization and format without examples.

**The Reality:** Examples teach AI exactly what you want, enforcing consistency across outputs.

**Your Task:** You want AI to categorize infrastructure alerts into severity categories. Currently, it might use inconsistent categories. Add 3 few-shot examples to establish your categorization system, then test it!

**Your Team's Categories:**
- **SECURITY:** Authentication, certificates, access control
- **AVAILABILITY:** Service health, uptime, connectivity
- **MAINTENANCE:** Backups, updates, routine tasks

In [ ]:
# ❌ BAD: No examples (generic, inconsistent categorization)
bad_messages = [
    {"role": "system", "content": "Categorize infrastructure alerts."},
    {"role": "user", "content": 'Categorize this alert: "Failed login attempts increased 400% on admin portal"'}
]

bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("WITHOUT EXAMPLES (Inconsistent Categorization):")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Add few-shot examples for alert categorization
# TODO: Uncomment and complete this section
# good_messages = [
#     {"role": "system", "content": "Categorize infrastructure alerts following the examples provided."},
#     
#     # Example 1
#     {"role": "user", "content": 'Categorize: "SSL certificate expires in 7 days for payment gateway"'},
#     {"role": "assistant", "content": "Category: SECURITY"},
#     
#     # Example 2
#     {"role": "user", "content": 'Categorize: "Load balancer health check failing for 2/8 backend servers"'},
#     {"role": "assistant", "content": "Category: AVAILABILITY"},
#     
#     # Example 3
#     {"role": "user", "content": 'Categorize: "Backup job completed with 3 file permission warnings"'},
#     {"role": "assistant", "content": "Category: MAINTENANCE"},
#     
#     # Now the actual request
#     {"role": "user", "content": 'Categorize: "Failed login attempts increased 400% on admin portal"'}
# ]
# 
# good_response = get_chat_completion(good_messages)
# print("=" * 70)
# print("WITH FEW-SHOT EXAMPLES (Consistent Categorization):")
# print("=" * 70)
# print(good_response)
# 
# print("\n💡 See how examples enforce your exact categories? The AI correctly identifies this as SECURITY!")

---

### ⛓️‍💥 Tactic 4: Chain-of-Thought Reasoning

**Guide AI through systematic step-by-step problem breakdown**

**Core Principle:** When faced with complex tasks like research, analysis, or problem-solving, having AI models break down problems into explicit, sequential steps dramatically improves performance. This technique, known as chain of thought (CoT) prompting, encourages the AI to work through problems methodically rather than jumping straight to conclusions, leading to more accurate and nuanced outputs.

Think of it like showing your work in math class—by making the intermediate reasoning steps visible, you catch errors, verify logic, and produce more reliable results.

**Why This Works:**
- **Accuracy:** Breaking problems into steps reduces errors, especially in math, logic, analysis, or complex tasks with multiple considerations
- **Coherence:** Structured thinking leads to more cohesive, well-organized responses  
- **Debugging:** Seeing the AI's thought process helps you pinpoint where prompts may be unclear or where reasoning breaks down
- **Transparency:** Makes AI decision-making auditable and explainable

**When to Use CoT:**
- Use for tasks that a human would need to think through carefully
- Examples: complex math, multi-step analysis, writing complex documents, decisions with many factors
- Especially valuable for: debugging workflows, architectural decisions, security analysis, test generation
- **Note:** Increased output length may impact latency, so use judiciously

**How to Implement CoT (from least to most complex):**

1. **Basic prompt:** Include "Think step-by-step" in your prompt
2. **Guided prompt:** Outline specific steps for the AI to follow in its thinking process  
3. **Structured prompt:** Use XML tags like `<thinking>` and `<answer>` to separate reasoning from the final answer

**Important:** Always have the AI output its thinking. Without outputting its thought process, no thinking occurs!

**Examples Below:** This section demonstrates three CoT patterns: forcing evaluation after solving, systematic multi-step code analysis, and a comprehensive review combining multiple techniques.

Test generation, code reviews, debugging workflows, architecture decisions, and security analysis are all critical areas where methodical analysis prevents missed issues.

*Reference: [Claude Documentation - Chain of Thought](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/chain-of-thought)*



#### **CoT Technique: Force AI to Analyze Before Diagnosing**

When troubleshooting production incidents, jumping to conclusions is dangerous. AI models exhibit similar behavior—they'll suggest a root cause prematurely if asked to diagnose immediately.

**The Problem:** Asking "What's the root cause?" directly causes AI to:
- Jump to obvious symptoms without deeper analysis
- Miss metric correlations and timeline patterns
- Confuse symptoms with actual root causes

**The Solution:** Force systematic analysis first:
1. Correlate all metrics with timeline
2. Identify primary vs. secondary symptoms
3. Trace causal relationships (A → B → C)
4. Only then diagnose root cause

**The Pattern:** *"Don't diagnose until you've systematically analyzed all metrics and their relationships."*

This technique is critical for incident response and performance debugging. Let's see it in action:

In [ ]:
# Example scenario: Root Cause Analysis (RCA) for production incidents
# We'll compare two approaches: immediate judgment vs. systematic analysis

problem = """
Problem: API errors spiked from 1% to 15% at 2 PM. CPU normal, memory up 30%, disk I/O high.

Step 1: Correlate metrics with timeline
- Error spike: 2 PM
- Memory increase: Started 1:55 PM
- Disk I/O spike: Started 2 PM
- CPU usage: Normal throughout

Step 2: Identify primary symptom
- Main issue: API errors (functional impact)
- Secondary: Resource usage changes (memory, disk I/O)

Step 3: Find root cause
- Disk I/O correlates exactly with error spike (both at 2 PM)
- Memory leak started earlier (1:55 PM), likely caused disk swapping
- CPU normal rules out compute bottleneck
- Causal chain: Memory leak → disk swapping → API timeouts

Conclusion / Root cause:
Memory leak triggered disk swapping at 2 PM, causing API request timeouts and error spike.
"""

new_incident = """
A web service's response time spiked from 200ms to 2000ms at 3 PM. 
CPU usage is normal, memory usage increased by 20%, and database query count doubled.
"""

# ❌ BAD APPROACH: Asking AI to diagnose immediately
# Risk: AI may jump to conclusions without systematic analysis
print("=" * 70)
print("BAD APPROACH: Immediate Diagnosis")
print("=" * 70)

bad_messages = [
    {
        "role": "system",
        "content": "You are a site reliability engineer."
    },
    {
        "role": "user",
        "content": f"""Diagnose this incident:

{new_incident}

What's the likely root cause?"""
    }
]

bad_response = get_chat_completion(bad_messages)
print(bad_response)
print("\n")

# ✅ GOOD APPROACH: Force AI to follow RCA methodology first, THEN diagnose
# Benefit: AI develops systematic understanding and can identify root cause accurately
print("=" * 70)
print("GOOD APPROACH: Systematic RCA Analysis")
print("=" * 70)

good_messages = [
    {
        "role": "system",
        "content": "You are a site reliability engineer skilled in root cause analysis."
    },
    {
        "role": "user",
        "content": f"""Study this RCA methodology example:

{problem}

Now apply the SAME 3-step format to diagnose this new incident:

{new_incident}

Use this exact structure:

Step 1: Correlate metrics with timeline
[List each metric with its timing]

Step 2: Identify primary symptom
[Distinguish main issue from secondary symptoms]

Step 3: Find root cause
[Analyze correlations, rule out non-factors, identify causal chain]

Conclusion / Root cause:
[Provide final diagnosis in one clear sentence]

Important: Follow the example's format exactly. Work through all steps before concluding."""
    }
]

good_response = get_chat_completion(good_messages)
print(good_response)

**📌 Key Takeaway: Systematic Analysis Beats Quick Diagnosis**

Notice the difference:
- **Bad approach:** AI jumps to conclusions without rigorous analysis
- **Good approach:** AI systematically correlates metrics, identifies causal chains, then diagnoses

**What Triggers Chain-of-Thought Reasoning?**

Any prompt structure that forces sequential, visible reasoning will trigger CoT. The more explicit your steps, the more systematic the analysis.

**5 Ways to Activate CoT:**

1. **Simple phrases:** "Think step-by-step" • "Before answering, first..." • "Show your reasoning"

2. **Structured instructions:**
   ```
   Step 1: Analyze inputs
   Step 2: Consider edge cases
   Step 3: Develop solution
   ```

3. **XML tags:** `<thinking>`, `<analysis>`, `<solution>` • Combine: "In <thinking> tags, work through this step-by-step"

4. **Numbered requirements:** "List three approaches and evaluate each" • "Identify all risks numbered 1-N"

5. **"Before X, First Y" patterns:** "Before diagnosing, first correlate all metrics" • "Before recommending fixes, first reproduce the issue"

#### **CoT Technique: Structured Analysis with XML Tags**

The previous example showed systematic RCA methodology. Now let's see another powerful CoT approach: **using XML tags to structure multi-step analysis**.

This example demonstrates **XML-structured Chain-of-Thought** where we:
- Use `<thinking>` tags to define explicit analysis steps
- Use `<analysis>` tags to document detailed findings
- Use `<solution>` tags to provide actionable recommendations
- Separate reasoning from conclusions for clear, auditable decision-making

This approach is particularly effective for:
- **Production troubleshooting** where you need clear reasoning trails
- **Complex debugging** with multiple potential root causes
- **Code reviews** requiring systematic analysis (security, performance, maintainability)
- **Incident post-mortems** where documentation and reproducibility matter

Let's see XML-structured CoT in action with a production bug investigation:

In [ ]:
# Chain-of-thought for systematic troubleshooting
system_message = """You are a senior engineer debugging production issues. Use systematic step-by-step analysis.

Structure your response using XML tags:

<thinking>
Step 1: Analyze the symptoms and reproduce the issue
Step 2: Examine relevant logs and stack traces
Step 3: Identify potential root causes
Step 4: Trace the execution flow to pinpoint the problem
</thinking>

<analysis>
Provide detailed findings for each step, explaining what you discovered and why it matters
</analysis>

<solution>
Recommend specific fixes with code changes and verification steps
</solution>"""

user_message = """
Debug this production issue:

**Symptoms:**
- User reports: "Items randomly disappear from shopping cart"
- Happens intermittently, ~10% of users affected
- No errors in application logs
- Issue started after deploying new caching layer

**Code:**
```python
# Cart service with Redis cache
class CartService:
    def add_item(self, user_id, item_id, quantity):
        cart = redis.get(f'cart:{user_id}') or []
        cart.append({'item': item_id, 'qty': quantity})
        redis.set(f'cart:{user_id}', cart, ex=3600)  # 1 hour TTL
        return cart
    
    def get_cart(self, user_id):
        return redis.get(f'cart:{user_id}') or []
    
    def remove_item(self, user_id, item_id):
        cart = redis.get(f'cart:{user_id}') or []
        cart = [item for item in cart if item['item'] != item_id]
        redis.set(f'cart:{user_id}', cart, ex=3600)
        return cart
```

**Environment:**
- 5 application servers behind load balancer
- Single Redis instance (no clustering)
- Average request rate: 500 req/sec
"""

chain_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

chain_response = get_chat_completion(chain_messages)
print("🔗 SYSTEMATIC TROUBLESHOOTING ANALYSIS:")
print(chain_response)

🚀 **Excellent!** The AI followed each step methodically, providing structured, comprehensive analysis of the production issue.

---

### 🎯 Try It Yourself: Chain-of-Thought Reasoning

**Common Misconception:** AI can analyze complex code and spot all issues instantly without systematic thinking.

**The Reality:** Step-by-step reasoning with structured outputs catches issues that instant analysis misses.

**Your Task:** Below is code with multiple issues. The first prompt asks for instant analysis. Fix it by adding chain-of-thought reasoning with XML tags:
1. Use `<security>` tags for security analysis
2. Use `<performance>` tags for performance review
3. Use `<quality>` tags for code quality assessment
4. Use `<recommendations>` tags for prioritized fixes

Compare which approach finds all the issues!

In [ ]:
# Code with multiple issues (SQL injection, connection pooling, error handling)
code_to_review = """
from flask import Flask, request, jsonify
import sqlite3

app = Flask(__name__)

@app.route('/user/<user_id>')
def get_user(user_id):
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM users WHERE id = {user_id}")
    user = cursor.fetchone()
    conn.close()
    
    if user:
        return jsonify({
            "id": user[0],
            "name": user[1], 
            "email": user[2]
        })
    else:
        return jsonify({"error": "User not found"}), 404
"""

context = """
This is a user lookup endpoint for a web application that serves user profiles.
The application handles 1000+ requests per minute during peak hours.
"""

# ❌ BAD: Instant analysis (might miss issues)
bad_messages = [
    {
        "role": "user",
        "content": f"Review this code:\n\n{code_to_review}\n\nContext: {context}"
    }
]

bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("INSTANT ANALYSIS (No Chain-of-Thought):")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Add chain-of-thought reasoning with XML tags
# TODO: Uncomment and complete this section
# good_messages = [
#     {
#         "role": "system",
#         "content": """You are a senior software engineer conducting a comprehensive code review.
# 
# Analyze the code systematically using XML tags:
# 
# <security>
# Identify security vulnerabilities with severity levels
# </security>
# 
# <performance>
# Analyze efficiency and optimization opportunities
# </performance>
# 
# <quality>
# Evaluate readability, maintainability, and best practices
# </quality>
# 
# <recommendations>
# Provide specific, prioritized fixes with code examples
# </recommendations>"""
#     },
#     {
#         "role": "user",
#         "content": f"""Review this code:
# 
# <code>
# {code_to_review}
# </code>
# 
# <context>
# {context}
# </context>
# 
# Perform a comprehensive code review using the structured XML format."""
#     }
# ]
# 
# good_response = get_chat_completion(good_messages)
# print("=" * 70)
# print("CHAIN-OF-THOUGHT ANALYSIS WITH XML TAGS:")
# print("=" * 70)
# print(good_response)
# 
# print("\n💡 The systematic approach catches SQL injection, connection pooling issues, and more!")

---

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); color: white; padding: 24px; border-radius: 12px; margin: 40px 0; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <div style="text-align: center; margin-bottom: 20px;">
    <h2 style="color: white; margin: 0; font-size: 1.8em; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">🍵 Suggested Break Point #2</h2>
    <p style="margin: 8px 0; font-size: 1.1em; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">~60 minutes elapsed • Halfway through!</p>
  </div>
  
  <div style="background: rgba(0,0,0,0.25); padding: 16px; border-radius: 8px; margin: 16px 0;">
    <p style="margin: 8px 0; font-size: 1.05em; font-weight: 600; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">✅ Completed (Tactics 0-4):</p>
    <ul style="margin: 8px 0; padding-left: 24px; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">
      <li>Clear Instructions & Role Prompting</li>
      <li>Structured Inputs with XML tags</li>
      <li>Few-Shot Examples for consistent styles</li>
      <li>Chain-of-Thought for systematic reasoning</li>
    </ul>
    <p style="margin: 12px 0 0 0; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">🎯 You've mastered 5 out of 8 tactics!</p>
  </div>
  
  <div style="background: rgba(0,0,0,0.25); padding: 16px; border-radius: 8px; margin: 16px 0;">
    <p style="margin: 8px 0; font-size: 1.05em; font-weight: 600; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">⏭️ Coming Next:</p>
    <ul style="margin: 8px 0; padding-left: 24px; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">
      <li>Tactic 5: Reference Citations (Ground responses in docs)</li>
      <li>Tactic 6: Prompt Chaining (Break complex tasks into steps)</li>
    </ul>
    <p style="margin: 12px 0 0 0; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">⏱️ Next section: ~30 minutes</p>
  </div>
  
  <div style="background: rgba(255,255,255,0.95); padding: 14px; border-radius: 8px; margin: 16px 0; text-align: center; color: #1e293b;">
    <p style="margin: 0; font-weight: bold; font-size: 1.1em; color: #1e293b;">📌 BOOKMARK TO RESUME:</p>
    <p style="margin: 8px 0 0 0; font-size: 1.15em; font-weight: bold; color: #0f172a;">"Tactic 5: Reference Citations"</p>
  </div>
  
  <p style="text-align: center; margin: 16px 0 0 0; font-size: 0.9em; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">
    💡 <em>Great progress! Consider taking a break before continuing with the final tactics.</em>
  </p>
</div>

---


---

### 📖 Tactic 5: Reference Citations

**Ground responses in actual documentation to reduce hallucinations**

**Core Principle:** When working with long documents or multiple reference materials, asking AI models to quote relevant parts of the documents first before carrying out tasks helps them cut through the "noise" and focus on pertinent information. This technique is especially powerful when working with extended context windows.

**Why This Works:**
- The AI identifies and focuses on relevant information before generating responses
- Citations make outputs verifiable and trustworthy
- Reduces hallucination by grounding responses in actual source material
- Makes it easy to trace conclusions back to specific code or documentation sections

**Best Practices for Long Context:**
- **Put longform data at the top:** Place long documents (~20K+ tokens) near the top of your prompt, above queries and instructions (can improve response quality by up to 30%)
- **Structure with XML tags:** Use `<documents>`, `<document>`, `<source>`, and `<document_content>` tags to organize multiple documents
- **Request quotes first:** Ask the AI to extract relevant quotes in `<quotes>` tags before generating the final response

**Working with External Documentation in IDEs:**

When using AI coding assistants like **GitHub Copilot**, **Claude Code**, or **OpenAI Codex**, you often need to reference documentation outside your codebase. Here's how to provide context effectively:

**1. Structure documents with XML tags (Recommended):**

Use this format for local files or external documentation. Keep a `docs/` folder with frequently-used references:
- `docs/api-conventions.md` - Your team's API standards
- `docs/external-apis/stripe.md` - Third-party API summaries  
- `docs/architecture.md` - System design decisions
- `docs/error-codes.md` - Standard error codes and handling

Then reference them in your prompts:

```xml
<documents>
  <document index="1">
    <source>docs/api-guide.md</source>
    <document_content>
    # Stripe Payment API
    POST /v1/payment_intents
    Creates a PaymentIntent object.
    
    Required fields:
    - amount (integer): Amount in cents
    - currency (string): 3-letter ISO code
    </document_content>
  </document>
  
  <document index="2">
    <source>docs/authentication.md</source>
    <document_content>
    Authentication uses Bearer tokens in the Authorization header.
    Example: Authorization: Bearer sk_test_abc123
    </document_content>
  </document>
</documents>

Based on the documentation above, implement a payment creation function.
```

**2. IDE-specific file shortcuts:**
- **GitHub Copilot Chat:** Use `#file:docs/api-guide.md` to quickly reference specific files
- **Claude Code:** Use `@filename` (e.g., `@docs/api-guide.md`) or open files in your editor as context
- **VS Code with Copilot:** Use `@workspace` or keep documentation files open in tabs

**3. Reference external URLs with key excerpts:**

For web documentation, extract and structure the relevant parts:

```xml
<document>
  <source>https://docs.stripe.com/api/payment_intents</source>
  <document_content>
  Key requirements:
  - Authentication: Bearer token in Authorization header  
  - amount must be in cents (e.g., $10.00 = 1000)
  - currency must be 3-letter ISO code
  - Successful response returns 200 with payment_intent object
  </document_content>
</document>
```

**Pro Tip:** Maintain local markdown summaries of frequently-used external APIs in your repo. This gives AI assistants grounded reference material and prevents hallucination of API details.

Code review with large codebases, documentation generation from source files, security audit reports, and analyzing API documentation all become more effective with proper citations.

*Reference: [Claude Documentation - Long Context Tips](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/long-context-tips)*

#### Example 1: Code Review with Multiple Files

Let's demonstrate how to structure multiple code files and ask the AI to extract relevant quotes before providing analysis:


In [ ]:
# Example: Multi-file code review with quote extraction
auth_service = """
class AuthService:
    def __init__(self, db_connection):
        self.db = db_connection
    
    def authenticate_user(self, username, password):
        # TODO: Add password hashing
        query = f"SELECT * FROM users WHERE username='{username}' AND password='{password}'"
        result = self.db.execute(query)
        return result.fetchone() is not None
    
    def create_session(self, user_id):
        session_id = str(uuid.uuid4())
        # Session expires in 24 hours
        expiry = datetime.now() + timedelta(hours=24)
        self.db.execute(f"INSERT INTO sessions VALUES ('{session_id}', {user_id}, '{expiry}')")
        return session_id
"""

user_controller = """
from flask import Flask, request, jsonify
from auth_service import AuthService

app = Flask(__name__)
auth = AuthService(db_connection)

@app.route('/login', methods=['POST'])
def login():
    username = request.json.get('username')
    password = request.json.get('password')
    
    if auth.authenticate_user(username, password):
        user_id = get_user_id(username)
        session_id = auth.create_session(user_id)
        return jsonify({'session_id': session_id, 'status': 'success'})
    else:
        return jsonify({'status': 'failed'}), 401
"""

# Structure the prompt with documents at the top, query at the bottom
messages = [
    {
        "role": "system",
        "content": "You are a senior security engineer reviewing code for vulnerabilities."
    },
    {
        "role": "user",
        "content": f"""<documents>
<document index="1">
<source>auth_service.py</source>
<document_content>
{auth_service}
</document_content>
</document>

<document index="2">
<source>user_controller.py</source>
<document_content>
{user_controller}
</document_content>
</document>
</documents>

Review the authentication code above for security vulnerabilities. 

First, extract relevant code quotes that demonstrate security issues and place them in <quotes> tags with the source file indicated.

Then, provide your security analysis in <analysis> tags, explaining each vulnerability and its severity.

Finally, provide specific remediation recommendations in <recommendations> tags."""
    }
]

response = get_chat_completion(messages)
print("🔒 SECURITY REVIEW WITH CITATIONS:")
print(response)


#### Example 2: API Documentation Analysis

Now let's analyze API documentation to extract specific information with citations:


In [ ]:
# Example: Analyzing API documentation with quote grounding
api_docs = """
# Payment API Documentation

## Authentication
All API requests require an API key passed in the `X-API-Key` header.
Rate limit: 1000 requests per hour per API key.

## Create Payment
POST /api/v2/payments

Creates a new payment transaction.

**Request Body:**
- amount (required, decimal): Payment amount in USD
- currency (optional, string): Currency code, defaults to "USD"
- customer_id (required, string): Customer identifier
- payment_method (required, string): One of: "card", "bank", "wallet"
- metadata (optional, object): Additional key-value pairs

**Rate Limit:** 100 requests per minute

**Response:**
{
  "payment_id": "pay_abc123",
  "status": "pending",
  "amount": 99.99,
  "created_at": "2024-01-15T10:30:00Z"
}

## Retrieve Payment
GET /api/v2/payments/{payment_id}

Retrieves details of a specific payment.

**Security Note:** Only returns payments belonging to the authenticated API key's account.

**Response Codes:**
- 200: Success
- 404: Payment not found
- 401: Invalid API key
"""

integration_question = """
I need to integrate payment processing into my e-commerce checkout flow.
The checkout needs to:
1. Create a payment when user clicks "Pay Now"
2. Handle USD and EUR currencies
3. Store order metadata with the payment
4. Check payment status after creation

What do I need to know from the API documentation?
"""

messages = [
    {
        "role": "system",
        "content": "You are a technical integration specialist helping developers implement APIs."
    },
    {
        "role": "user",
        "content": f"""<documents>
<document index="1">
<source>payment_api_docs.md</source>
<document_content>
{api_docs}
</document_content>
</document>
</documents>

<integration_requirements>
{integration_question}
</integration_requirements>

First, find and quote the relevant sections from the API documentation that address the integration requirements. Place these quotes in <quotes> tags with the section name indicated.

Then, provide a step-by-step integration guide in <integration_guide> tags that references the quoted documentation."""
    }
]

response = get_chat_completion(messages)
print("📚 API INTEGRATION GUIDE WITH CITATIONS:")
print(response)


#### Key Takeaways: Reference Citations

**Best Practices Demonstrated:**
1. **Document Structure:** Used `<documents>` and `<document>` tags with `<source>` and `<document_content>` metadata
2. **Documents First:** Placed all reference materials at the top of the prompt, before the query
3. **Quote Extraction:** Asked AI to extract relevant quotes first, then perform analysis
4. **Structured Output:** Used XML tags like `<quotes>`, `<analysis>`, and `<recommendations>` to organize responses


---

### 🎯 Try It Yourself: Reference Citations

**Common Misconception:** AI can accurately implement APIs from general knowledge without documentation.

**The Reality:** Without reference citations, AI invents plausible-sounding but incorrect details. Quote extraction grounds responses in actual documentation.

**Documentation Setup:**

This exercise uses actual documentation files in the `docs/` directory:
```
module-02-fundamentals/
├── docs/
│   ├── stripe-api-guide.md       # API endpoint details, required fields
│   └── stripe-authentication.md  # Authentication methods, security
└── module2.ipynb
```

**Your Task:** Compare two approaches to the same task:

**Bad approach (without citations):**
- Vague prompt with no documentation
- AI relies on general knowledge about Stripe
- Results in hallucinations (wrong fields, wrong endpoints, wrong auth)

**Good approach (with citations):**
1. Load documentation from actual files: `docs/stripe-api-guide.md`, `docs/stripe-authentication.md`
2. Structure using XML tags: `<documents>`, `<document>`, `<source>`, `<document_content>`
3. Request quote extraction in `<quotes>` tags first
4. AI implements using ONLY the quoted information

**What you'll learn:** See programmatic validation detect hallucinations in the bad response, then watch them disappear with proper citations!

In [ ]:
# Load Payment API Documentation from actual files
with open('docs/stripe-api-guide.md', 'r') as f:
    api_guide = f.read()

with open('docs/stripe-authentication.md', 'r') as f:
    authentication_guide = f.read()

# Helper function to validate responses against documentation
def validate_response(response, label):
    """Check if response matches documentation (detects hallucinations)"""
    required_fields = ["amount", "currency", "payment_method"]
    endpoint = "/v1/payment_intents"
    
    print(f"\n{'='*70}")
    print(f"🔍 HALLUCINATION CHECK - {label}:")
    print('='*70)
    
    # Check key requirements
    checks = {
        "Has required fields (amount, currency, payment_method)": all(field in response.lower() for field in required_fields),
        "Uses correct endpoint /v1/payment_intents": endpoint in response,
        "Uses Bearer token authentication": "bearer" in response.lower(),
    }
    
    print("\nValidating against documentation:")
    for check, passed in checks.items():
        print(f"  {'✅' if passed else '❌'} {check}")
    
    # Detect common hallucinations
    hallucinations = []
    if any(field in response.lower() for field in ["customer_name", "card_number", "stripe_key", "api_secret"]):
        hallucinations.append("Invents fields not in docs")
    if "x-api-key" in response.lower() or ("api key" in response.lower() and "bearer" not in response.lower()):
        hallucinations.append("Wrong authentication method")
    
    if hallucinations:
        print(f"\n⚠️  Hallucinations detected: {', '.join(hallucinations)}")
        return len(hallucinations)
    else:
        print("\n✅ No hallucinations detected!")
        return 0

# ❌ BAD: Vague request without structured citations
bad_messages = [{
    "role": "user",
    "content": """Create a Python function to process Stripe payments. 
    
The function should:
- Accept payment details like amount, card info, customer details
- Handle authentication with Stripe API
- Return the payment status

Make it production-ready with proper error handling."""
}]

print("="*70)
print("WITHOUT REFERENCE CITATIONS:")
print("="*70)
bad_response = get_chat_completion(bad_messages)
print(bad_response)
bad_count = validate_response(bad_response, "WITHOUT CITATIONS")

print("\n" + "="*70 + "\n")

# ✅ YOUR TURN: Use multi-document XML structure with quote extraction
# TODO: Uncomment and complete
# good_messages = [{
#     "role": "user",
#     "content": f"""<documents>
#   <document index="1">
#     <source>docs/stripe-api-guide.md</source>
#     <document_content>
#       {api_guide}
#     </document_content>
#   </document>
#   
#   <document index="2">
#     <source>docs/stripe-authentication.md</source>
#     <document_content>
#       {authentication_guide}
#     </document_content>
#   </document>
# </documents>
# 
# Task: Create a Python function to process Stripe payments.
# 
# The function should:
# - Accept payment details like amount, card info, customer details
# - Handle authentication with Stripe API
# - Return the payment status
# 
# Make it production-ready with proper error handling.
# 
# Step 1: Extract relevant quotes from the documentation above.
# In <quotes> tags, extract:
# - Required API fields from document 1
# - Authentication format from document 2
# - Correct endpoint from document 1
# 
# Step 2: Using ONLY the quoted information, provide implementation in <code> tags."""
# }]
# 
# print("="*70)
# print("WITH REFERENCE CITATIONS:")
# print("="*70)
# good_response = get_chat_completion(good_messages)
# print(good_response)
# good_count = validate_response(good_response, "WITH CITATIONS")
# 
# # Show comparison
# print(f"\n{'='*70}")
# print("📊 COMPARISON:")
# print('='*70)
# print(f"Without citations: {bad_count} hallucination(s) ⚠️")
# print(f"With citations: {good_count} hallucination(s) ✅")
# print("\n💡 Without documentation, AI invents plausible but incorrect details!")
# print("   With citations, AI stays grounded in actual API requirements.")

---

### 🔗 Tactic 6: Prompt Chaining

**Break complex tasks into sequential workflows**

**Core Principle:** When working with complex tasks, AI models can sometimes drop the ball if you try to handle everything in a single prompt. Prompt chaining breaks down complex tasks into smaller, manageable subtasks, where each subtask gets the AI's full attention.

**Why Chain Prompts:**
- **Accuracy:** Each subtask gets full attention, reducing errors
- **Clarity:** Simpler subtasks mean clearer instructions and outputs
- **Traceability:** Easily pinpoint and fix issues in your prompt chain
- **Focus:** Each link in the chain gets the AI's complete concentration

**When to Chain Prompts:**
Use prompt chaining for multi-step tasks like:
- Research synthesis and document analysis
- Iterative content creation
- Multiple transformations or citations
- Code generation → Review → Refactoring workflows

**How to Chain Prompts:**
1. **Identify subtasks:** Break your task into distinct, sequential steps
2. **Structure with XML:** Use XML tags to pass outputs between prompts
3. **Single-task goal:** Each subtask should have one clear objective
4. **Iterate:** Refine subtasks based on performance

**Common Software Development Workflows:**
- **Code Review Pipeline:** Extract code → Analyze issues → Propose fixes → Generate tests
- **Documentation Generation:** Analyze code → Extract docstrings → Format → Review
- **Refactoring Workflow:** Identify patterns → Suggest improvements → Generate refactored code → Validate
- **Testing Pipeline:** Analyze function → Generate test cases → Create assertions → Review coverage
- **Debugging Chain:** Reproduce issue → Analyze root cause → Suggest fixes → Verify solution

**Debugging Tip:** If the AI misses a step or performs poorly, isolate that step in its own prompt. This lets you fine-tune problematic steps without redoing the entire task.

Complex code reviews, multi-stage refactoring, comprehensive test generation, and architectural analysis all benefit from prompt chaining to ensure nothing is missed.

*Reference: [Claude Documentation - Chain Complex Prompts](https://docs.claude.com/en/docs/build-with-claude/prompt-engineering/chain-prompts)*


#### Example 1: Code Review with Prompt Chaining

Let's demonstrate a 3-step prompt chain for comprehensive code review:
1. **Step 1:** Analyze code for issues
2. **Step 2:** Review the analysis for completeness
3. **Step 3:** Generate final recommendations with fixes


In [ ]:
# Prompt Chain Example: Code Review Pipeline
code_to_review = """
def process_user_data(user_input):
    # Process user registration data
    data = eval(user_input)  # Parse input
    
    username = data['username']
    email = data['email']
    password = data['password']
    
    # Save to database
    query = f"INSERT INTO users (username, email, password) VALUES ('{username}', '{email}', '{password}')"
    db.execute(query)
    
    # Send welcome email
    send_email(email, f"Welcome {username}!")
    
    return {"status": "success", "user": username}
"""

# STEP 1: Analyze code for issues
print("=" * 60)
print("STEP 1: Initial Code Analysis")
print("=" * 60)

step1_messages = [
    {
        "role": "system",
        "content": "You are a senior code reviewer specializing in security and best practices."
    },
    {
        "role": "user",
        "content": f"""Analyze this Python function for issues:

<code>
{code_to_review}
</code>

Identify all security vulnerabilities, code quality issues, and potential bugs.
Provide your analysis in <analysis> tags with specific line references."""
    }
]

analysis = get_chat_completion(step1_messages)
print(analysis)
print("\n")

# STEP 2: Review the analysis for completeness
print("=" * 60)
print("STEP 2: Review Analysis for Completeness")
print("=" * 60)

step2_messages = [
    {
        "role": "system",
        "content": "You are a principal engineer reviewing a code analysis. Check for completeness and accuracy."
    },
    {
        "role": "user",
        "content": f"""Here is a code analysis from a code reviewer:

<original_code>
{code_to_review}
</original_code>

<initial_analysis>
{analysis}
</initial_analysis>

Review this analysis and:
1. Verify all issues are correctly identified
2. Check if any critical issues were missed
3. Rate the severity of each issue (Critical/High/Medium/Low)

Provide feedback in <review> tags."""
    }
]

review = get_chat_completion(step2_messages)
print(review)
print("\n")

# STEP 3: Generate final recommendations with code fixes
print("=" * 60)
print("STEP 3: Final Recommendations and Code Fixes")
print("=" * 60)

step3_messages = [
    {
        "role": "system",
        "content": "You are a senior developer providing actionable solutions."
    },
    {
        "role": "user",
        "content": f"""Based on the code analysis and review, provide final recommendations:

<original_code>
{code_to_review}
</original_code>

<analysis>
{analysis}
</analysis>

<review>
{review}
</review>

Provide:
1. A prioritized list of fixes in <fixes> tags
2. The complete refactored code in <refactored_code> tags
3. Brief explanation of key changes in <explanation> tags"""
    }
]

final_recommendations = get_chat_completion(step3_messages)
print(final_recommendations)


#### Example 2: Test Generation with Prompt Chaining

Now let's create a chain for comprehensive test generation:
1. **Step 1:** Analyze function to identify test scenarios
2. **Step 2:** Generate test cases based on scenarios  
3. **Step 3:** Review and enhance test coverage


In [ ]:
# Prompt Chain Example: Test Generation Pipeline
function_to_test = """
def calculate_discount(price, discount_percent, customer_tier='standard'):
    \"\"\"
    Calculate final price after applying discount.
    
    Args:
        price: Original price (must be positive)
        discount_percent: Discount percentage (0-100)
        customer_tier: Customer tier ('standard', 'premium', 'vip')
    
    Returns:
        Final price after discount and tier bonus
    \"\"\"
    if price < 0:
        raise ValueError("Price cannot be negative")
    
    if discount_percent < 0 or discount_percent > 100:
        raise ValueError("Discount must be between 0 and 100")
    
    # Apply base discount
    discounted_price = price * (1 - discount_percent / 100)
    
    # Apply tier bonus
    tier_bonuses = {'standard': 0, 'premium': 5, 'vip': 10}
    if customer_tier not in tier_bonuses:
        raise ValueError(f"Invalid tier: {customer_tier}")
    
    tier_bonus = tier_bonuses[customer_tier]
    final_price = discounted_price * (1 - tier_bonus / 100)
    
    return round(final_price, 2)
"""

# STEP 1: Analyze function and identify test scenarios
print("=" * 60)
print("STEP 1: Analyze Function and Identify Test Scenarios")
print("=" * 60)

step1_messages = [
    {
        "role": "system",
        "content": "You are a QA engineer analyzing code for test coverage."
    },
    {
        "role": "user",
        "content": f"""Analyze this function and identify all test scenarios needed:

<function>
{function_to_test}
</function>

Identify and categorize test scenarios:
1. Happy path scenarios
2. Edge cases
3. Error cases
4. Boundary conditions

Provide your analysis in <test_scenarios> tags."""
    }
]

test_scenarios = get_chat_completion(step1_messages)
print(test_scenarios)
print("\n")

# STEP 2: Generate test cases based on scenarios
print("=" * 60)
print("STEP 2: Generate Test Cases")
print("=" * 60)

step2_messages = [
    {
        "role": "system",
        "content": "You are a test automation engineer. Write pytest test cases."
    },
    {
        "role": "user",
        "content": f"""Based on these test scenarios, generate pytest test cases:

<function>
{function_to_test}
</function>

<test_scenarios>
{test_scenarios}
</test_scenarios>

Generate complete, executable pytest test cases in <test_code> tags.
Include assertions, test data, and descriptive test names."""
    }
]

test_code = get_chat_completion(step2_messages)
print(test_code)
print("\n")

# STEP 3: Review and enhance test coverage
print("=" * 60)
print("STEP 3: Review Test Coverage and Suggest Enhancements")
print("=" * 60)

step3_messages = [
    {
        "role": "system",
        "content": "You are a principal QA engineer reviewing test coverage."
    },
    {
        "role": "user",
        "content": f"""Review this test suite for completeness:

<function>
{function_to_test}
</function>

<test_scenarios>
{test_scenarios}
</test_scenarios>

<generated_tests>
{test_code}
</generated_tests>

Evaluate:
1. Are all scenarios covered?
2. Are there any missing edge cases?
3. Is the test data comprehensive?
4. Estimate coverage percentage

Provide:
- Coverage assessment in <coverage_review> tags
- Any additional test cases needed in <additional_tests> tags"""
    }
]

coverage_review = get_chat_completion(step3_messages)
print(coverage_review)


#### Key Takeaways: Prompt Chaining

**What We Demonstrated:**

**Example 1: Code Review Chain**
- **Step 1:** Initial analysis identifies security vulnerabilities and code quality issues
- **Step 2:** Principal engineer validates the analysis and adds severity ratings
- **Step 3:** Generates actionable fixes and refactored code

**Example 2: Test Generation Chain**
- **Step 1:** Analyzes function to identify all necessary test scenarios
- **Step 2:** Generates complete pytest test cases with proper structure
- **Step 3:** Reviews coverage and suggests additional tests for completeness

**Why Chaining Works Better Than Single Prompts:**
- **Focused attention:** Each step handles one specific task without distraction
- **Quality control:** Later steps can review and enhance earlier outputs
- **Iterative refinement:** Each link improves the overall result
- **Easier debugging:** Problems can be isolated to specific steps

**Best Practices Demonstrated:**
1. **Pass context forward:** Each step receives relevant outputs from previous steps
2. **Use XML tags:** Structured tags (`<analysis>`, `<review>`, `<test_code>`) organize data flow
3. **Clear objectives:** Each step has one specific, measurable goal
4. **Role specialization:** Different expert personas for different steps

**Real-World Applications:**
- **Multi-stage refactoring:** Analyze → Plan → Refactor → Validate → Document
- **Comprehensive security audits:** Scan → Analyze → Prioritize → Generate fixes → Verify
- **API development:** Design schema → Generate code → Create tests → Write docs → Review
- **Database migrations:** Analyze schema → Generate migration → Create rollback → Test → Deploy
- **CI/CD pipeline generation:** Analyze project → Design workflow → Generate config → Add tests → Optimize

**Pro Tip:** You can also create **self-correction chains** where the AI reviews its own work! Just pass the output back with a review prompt to catch errors and refine results.


---

### 🎯 Try It Yourself: Prompt Chaining

**Common Misconception:** AI can handle multiple complex tasks in one prompt just as well as breaking them into steps.

**The Reality:** Chaining gives each task full attention, dramatically improving quality and reducing errors.

**Your Task:** Below is an overwhelming single prompt that asks for code review, fixes, tests, and documentation all at once. Break it into a 3-step chain:
1. **Step 1:** Analyze code for issues
2. **Step 2:** Generate fixed code based on analysis
3. **Step 3:** Create tests for the fixed code

Compare the single-prompt chaos with the systematic chain!

In [ ]:
# Code with issues
problematic_code = """
def process_payment(amount, card_number):
    if amount > 0:
        charge_card(card_number, amount)
        return "success"
"""

# ❌ BAD: Everything at once (overwhelming)
bad_messages = [{
    "role": "user",
    "content": f"""Review this code, fix all issues, write tests, and add documentation:
{problematic_code}"""
}]
bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("SINGLE PROMPT (Everything at once):")
print("=" * 70)
print(bad_response[:500] + "..." if len(bad_response) > 500 else bad_response)
print("\n")

# ✅ YOUR TURN: Create a 3-step chain
# TODO: Uncomment and complete
# # STEP 1: Analyze
# step1 = [{"role": "user", "content": f"Analyze this code for issues:\n{problematic_code}\nProvide analysis in <analysis> tags."}]
# analysis = get_chat_completion(step1)
# print("STEP 1 - Analysis:"); print(analysis); print("\n")
# 
# # STEP 2: Fix based on analysis
# step2 = [{"role": "user", "content": f"<code>{problematic_code}</code>\n<analysis>{analysis}</analysis>\nGenerate fixed code in <fixed_code> tags."}]
# fixed = get_chat_completion(step2)
# print("STEP 2 - Fixed Code:"); print(fixed); print("\n")
# 
# # STEP 3: Generate tests
# step3 = [{"role": "user", "content": f"Create pytest tests for:\n{fixed}\nInclude edge cases."}]
# tests = get_chat_completion(step3)
# print("STEP 3 - Tests:"); print(tests)
# 
# print("\n💡 Each step got full attention - notice the thoroughness!")

---

<div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); color: white; padding: 24px; border-radius: 12px; margin: 40px 0; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <div style="text-align: center; margin-bottom: 20px;">
    <h2 style="color: white; margin: 0; font-size: 1.8em; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">🧃 Suggested Break Point #3</h2>
    <p style="margin: 8px 0; font-size: 1.1em; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">~90 minutes elapsed • Almost there!</p>
  </div>
  
  <div style="background: rgba(0,0,0,0.25); padding: 16px; border-radius: 8px; margin: 16px 0;">
    <p style="margin: 8px 0; font-size: 1.05em; font-weight: 600; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">✅ Completed (Tactics 0-6):</p>
    <ul style="margin: 8px 0; padding-left: 24px; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">
      <li>Clear Instructions, Role Prompting & Structured Inputs</li>
      <li>Few-Shot Examples & Chain-of-Thought</li>
      <li>Reference Citations for grounded responses</li>
      <li>Prompt Chaining for complex workflows</li>
    </ul>
    <p style="margin: 12px 0 0 0; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">🎯 You've mastered 7 out of 8 tactics!</p>
  </div>
  
  <div style="background: rgba(0,0,0,0.25); padding: 16px; border-radius: 8px; margin: 16px 0;">
    <p style="margin: 8px 0; font-size: 1.05em; font-weight: 600; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">⏭️ Final Sprint:</p>
    <ul style="margin: 8px 0; padding-left: 24px; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">
      <li>Tactic 7: LLM-as-Judge (Create evaluation rubrics)</li>
      <li>Tactic 8: Inner Monologue (Clean outputs)</li>
      <li>Hands-On Practice Activities (Apply what you learned)</li>
    </ul>
    <p style="margin: 12px 0 0 0; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">⏱️ Remaining time: ~30-40 minutes</p>
  </div>
  
  <div style="background: rgba(255,255,255,0.95); padding: 14px; border-radius: 8px; margin: 16px 0; text-align: center; color: #1e293b;">
    <p style="margin: 0; font-weight: bold; font-size: 1.1em; color: #1e293b;">📌 BOOKMARK TO RESUME:</p>
    <p style="margin: 8px 0 0 0; font-size: 1.15em; font-weight: bold; color: #0f172a;">"Tactic 7: LLM-as-Judge"</p>
  </div>
  
  <p style="text-align: center; margin: 16px 0 0 0; font-size: 0.9em; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">
    💡 <em>You're in the home stretch! Take a quick break before the final tactics.</em>
  </p>
</div>

---


---

### ⚖️ Tactic 7: LLM-as-Judge

**Create evaluation rubrics and self-critique loops**

**Core Principle:** One of the most powerful patterns in prompt engineering is using an AI model as a judge or critic to evaluate and improve outputs. This creates a self-improvement loop where the AI reviews, critiques, and refines work—either its own outputs or those from other sources.

**Why Use LLM-as-Judge:**
- **Quality assurance:** Catch errors, inconsistencies, and areas for improvement
- **Objective evaluation:** Get unbiased assessment based on specific criteria
- **Iterative refinement:** Continuously improve outputs through multiple review cycles
- **Scalable review:** Automate code reviews, documentation checks, and quality audits

**When to Use LLM-as-Judge:**
- Code review and quality assessment
- Evaluating multiple solution approaches
- Grading or scoring responses against rubrics
- Providing constructive feedback on technical writing
- Testing and validation of AI-generated content
- Comparing different implementations

**How to Implement:**
1. **Define clear criteria:** Specify what makes a good/bad output
2. **Provide rubrics:** Give the judge specific evaluation dimensions
3. **Request structured feedback:** Ask for scores, ratings, or categorized feedback
4. **Include examples:** Show what excellent vs. poor outputs look like
5. **Iterate:** Use feedback to improve and re-evaluate

Automated code reviews, architecture decision validation, test coverage assessment, documentation quality checks, and comparing multiple implementation approaches all benefit from LLM-as-Judge.

*Reference: This technique combines elements from evaluation frameworks and self-critique patterns used in production AI systems.*


#### Example 1: Code Quality Judge

Let's use AI as a judge to evaluate and compare two different implementations:


In [ ]:
# Example: LLM as Judge - Comparing Two Implementations
implementation_a = """
def find_duplicates(items):
    duplicates = []
    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            if items[i] == items[j] and items[i] not in duplicates:
                duplicates.append(items[i])
    return duplicates
"""

implementation_b = """
def find_duplicates(items):
    from collections import Counter
    counts = Counter(items)
    return [item for item, count in counts.items() if count > 1]
"""

print("=" * 70)
print("LLM AS JUDGE: Comparing Implementations")
print("=" * 70)

judge_messages = [
    {
        "role": "system",
        "content": """You are a senior software engineer acting as an impartial code judge.
        
Evaluate code based on these criteria:
1. Time Complexity (weight: 30%)
2. Space Complexity (weight: 20%)
3. Readability (weight: 25%)
4. Maintainability (weight: 15%)
5. Edge Case Handling (weight: 10%)

Provide:
- Scores (0-10) for each criterion
- Overall weighted score
- Pros and cons for each implementation
- Final recommendation"""
    },
    {
        "role": "user",
        "content": f"""Compare these two implementations of a function that finds duplicate items in a list:

<implementation_a>
{implementation_a}
</implementation_a>

<implementation_b>
{implementation_b}
</implementation_b>

Evaluate both implementations using the criteria provided. Structure your response with:
1. <evaluation_a> tags for Implementation A analysis
2. <evaluation_b> tags for Implementation B analysis
3. <comparison> tags for side-by-side comparison
4. <recommendation> tags for final verdict"""
    }
]

judge_response = get_chat_completion(judge_messages)
print(judge_response)


#### Example 2: Self-Critique and Improvement Loop

Now let's create an improvement loop where AI generates code, critiques it, and then improves it:


In [ ]:
# Example: Self-Critique and Improvement Loop
requirement = "Create a function that validates and sanitizes user input for a SQL query"

# STEP 1: Generate initial solution
print("=" * 70)
print("STEP 1: Generate Initial Solution")
print("=" * 70)

generate_messages = [
    {
        "role": "system",
        "content": "You are a Python developer. Generate code solutions."
    },
    {
        "role": "user",
        "content": f"""{requirement}

Provide your implementation in <code> tags."""
    }
]

initial_code = get_chat_completion(generate_messages)
print(initial_code)
print("\n")

# STEP 2: Critique the solution
print("=" * 70)
print("STEP 2: Critique the Solution")
print("=" * 70)

critique_messages = [
    {
        "role": "system",
        "content": """You are a security-focused code reviewer. 
        
Evaluate code for:
- Security vulnerabilities
- Best practices
- Error handling
- Edge cases
- Code quality

Provide brutally honest feedback with specific issues and severity levels."""
    },
    {
        "role": "user",
        "content": f"""Requirement: {requirement}

Initial implementation:
{initial_code}

Critique this implementation. Identify all issues, rate severity (Critical/High/Medium/Low), and suggest specific improvements.

Structure your response:
<critique>Your detailed critique</critique>
<issues>List of specific issues with severity</issues>
<suggestions>Actionable improvement suggestions</suggestions>"""
    }
]

critique = get_chat_completion(critique_messages)
print(critique)
print("\n")

# STEP 3: Improve based on critique
print("=" * 70)
print("STEP 3: Improved Implementation")
print("=" * 70)

improve_messages = [
    {
        "role": "system",
        "content": "You are a senior Python developer who learns from feedback."
    },
    {
        "role": "user",
        "content": f"""Requirement: {requirement}

Original implementation:
{initial_code}

Critique received:
{critique}

Create an improved implementation that addresses ALL the issues raised in the critique.
Provide the improved code in <improved_code> tags and explain key changes in <changes> tags."""
    }
]

improved_code = get_chat_completion(improve_messages)
print(improved_code)


#### Key Takeaways: LLM-as-Judge

**What We Demonstrated:**

**Example 1: Code Quality Judge**
- Defined clear evaluation criteria with weights
- Provided structured rubrics for assessment
- Got objective comparison of two implementations
- Received scored evaluation with pros/cons and recommendation

**Example 2: Self-Critique and Improvement Loop**
- **Step 1:** Generated initial code solution
- **Step 2:** Used AI as brutal critic to identify issues
- **Step 3:** Improved code based on critique feedback
- Created a self-improvement cycle

**Benefits of LLM-as-Judge:**

1. **Objective Evaluation:**
   - Unbiased assessment based on defined criteria
   - Consistent scoring across multiple evaluations
   - Reduces human bias in code reviews

2. **Continuous Improvement:**
   - Iterative refinement through critique loops
   - Learn from mistakes and feedback
   - Progressive quality enhancement

3. **Scalable Reviews:**
   - Automate repetitive evaluation tasks
   - Handle multiple implementations simultaneously
   - Save senior engineers' time for complex decisions

4. **Structured Feedback:**
   - Clear, actionable improvement suggestions
   - Severity ratings for prioritization
   - Specific examples and recommendations

**Real-World Applications:**

- **Automated Code Reviews:** Evaluate PRs against coding standards before human review
- **Architecture Decisions:** Compare multiple design approaches objectively
- **Test Quality Assessment:** Evaluate test coverage and edge case handling
- **Documentation Quality:** Grade documentation completeness and clarity
- **API Design Review:** Compare REST vs GraphQL implementations
- **Performance Optimization:** Evaluate before/after optimization attempts
- **Security Audits:** Systematic vulnerability assessment with severity ratings

**Implementation Patterns:**

```python
# Pattern 1: Single evaluation
judge_prompt = """
Evaluate [OUTPUT] based on:
1. Criterion A (weight: X%)
2. Criterion B (weight: Y%)

Provide scores and recommendation.
"""

# Pattern 2: Comparative evaluation
judge_prompt = """
Compare [OPTION_A] and [OPTION_B] against:
- Criteria 1
- Criteria 2
- Criteria 3

Recommend the better option with justification.
"""

# Pattern 3: Self-improvement loop
1. Generate solution
2. Critique solution (AI as judge)
3. Improve based on critique
4. (Optional) Re-evaluate improvement
```

**Pro Tips:**
- **Define clear rubrics:** Specific criteria produce better judgments
- **Use weighted scoring:** Prioritize what matters most
- **Request examples:** Ask for specific code snippets in feedback
- **Iterate multiple times:** Don't stop at first critique
- **Combine with other tactics:** Use with prompt chaining for multi-stage reviews


---

### 🎯 Try It Yourself: LLM-as-Judge

**Common Misconception:** AI comparisons are just subjective opinions without clear reasoning.

**The Reality:** Weighted evaluation rubrics produce objective, actionable assessments.

**Your Task:** You have two implementations below. The bad prompt just asks "which is better?" Fix it by:
1. Creating specific evaluation criteria
2. Adding weights to each criterion (e.g., Security 40%, Performance 30%, Readability 30%)
3. Requesting scores and structured comparison

See how rubrics transform vague opinions into actionable insights!

In [ ]:
# Two implementations to compare
impl_a = "def hash_pwd(p): return hashlib.md5(p.encode()).hexdigest()"
impl_b = "def hash_pwd(p): return bcrypt.hashpw(p.encode(), bcrypt.gensalt())"

# ❌ BAD: Vague comparison request
bad_messages = [{"role": "user", "content": f"Which is better?\nA: {impl_a}\nB: {impl_b}"}]
bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("WITHOUT RUBRIC (Vague opinion):")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Create evaluation rubric
# TODO: Uncomment and complete
# good_messages = [{
#     "role": "system",
#     "content": """You are a code quality judge. Evaluate based on:
# - Security (40%): Resistance to attacks, proper crypto
# - Performance (30%): Speed, resource usage
# - Readability (30%): Clear, maintainable
# 
# Provide scores 0-10 for each, calculate weighted total, recommend best option."""
#     },
#     {
#         "role": "user",
#         "content": f"""Compare these password hashing implementations:
# 
# <implementation_a>
# {impl_a}
# </implementation_a>
# 
# <implementation_b>
# {impl_b}
# </implementation_b>
# 
# Provide:
# - Scores for each criterion
# - Weighted total scores
# - Recommendation with justification"""
#     }
# ]
# good_response = get_chat_completion(good_messages)
# print("=" * 70)
# print("WITH RUBRIC (Objective assessment):")
# print("=" * 70)
# print(good_response)
# print("\n💡 Rubric provides clear, actionable comparison with reasoning!")

---

### 🤫 Tactic 8: Inner Monologue

**Separate reasoning from clean final outputs**

**Core Principle:** The Inner Monologue technique guides AI models to articulate their thought process internally before delivering a final response, effectively "hiding" the reasoning steps from the end user. This is particularly useful when you want the benefits of chain-of-thought reasoning without exposing the intermediate thinking to users.

**Why Use Inner Monologue:**
- **Cleaner output:** Users see only the final answer, not the reasoning steps
- **Better reasoning:** The AI still benefits from step-by-step thinking internally
- **Professional presentation:** Provides concise, polished responses without verbose explanations
- **Flexible control:** You decide what to show and what to keep internal

**When to Use Inner Monologue:**
- Customer-facing applications where clean responses are important
- API responses that need to be concise
- Documentation generation where only conclusions matter
- Code generation where you want the code, not the thought process
- Production systems where token efficiency is critical

**How to Implement:**
1. **Instruct internal thinking:** Tell the AI to think through the problem internally
2. **Separate reasoning from output:** Use tags like `<thinking>` for internal reasoning and `<output>` for final results
3. **Extract final result:** Parse only the `<output>` section for user-facing display
4. **Optional logging:** Store the `<thinking>` section for debugging or quality assurance

Code generation tools, automated PR reviews, documentation generators, and customer-facing chatbots all benefit from intelligent responses without exposing the AI's reasoning process.


#### Example 1: Code Generation with Hidden Reasoning

Let's compare code generation with and without inner monologue:


In [ ]:
# Example 1: WITHOUT Inner Monologue (verbose response)
print("=" * 70)
print("WITHOUT INNER MONOLOGUE (Verbose)")
print("=" * 70)

without_monologue = [
    {
        "role": "system",
        "content": "You are a Python developer helping with code generation."
    },
    {
        "role": "user",
        "content": """Create a function that validates email addresses using regex. 
It should check for proper format and common email providers."""
    }
]

response_verbose = get_chat_completion(without_monologue)
print(response_verbose)
print("\n")

# Example 1: WITH Inner Monologue (clean output)
print("=" * 70)
print("WITH INNER MONOLOGUE (Clean Output Only)")
print("=" * 70)

with_monologue = [
    {
        "role": "system",
        "content": """You are a Python developer. When solving problems:
1. Think through the requirements internally in <thinking> tags
2. Provide only the final code in <output> tags
3. Keep the output clean and production-ready"""
    },
    {
        "role": "user",
        "content": """Create a function that validates email addresses using regex. 
It should check for proper format and common email providers.

Think through the requirements internally, then provide only the final code."""
    }
]

response_clean = get_chat_completion(with_monologue)
print(response_clean)

# Extract only the output section (simulating production use)
import re
output_match = re.search(r'<output>(.*?)</output>', response_clean, re.DOTALL)
if output_match:
    print("\n" + "=" * 70)
    print("EXTRACTED FOR USER (Production Output)")
    print("=" * 70)
    print(output_match.group(1).strip())


#### Key Takeaways: Inner Monologue

**What We Demonstrated:**

**Example 1: Code Generation**
- **Without inner monologue:** AI provides verbose explanations mixed with code
- **With inner monologue:** AI thinks internally in `<thinking>` tags, outputs clean code in `<output>` tags
- **Production use:** Extract only the `<output>` section for user-facing applications

**Example 2: Bug Analysis**
- AI analyzes the bug internally (division by zero for empty list)
- Provides concise, actionable fix without lengthy explanation
- Perfect for automated bug-fixing tools or PR comments

**Benefits of Inner Monologue:**

1. **Best of Both Worlds:**
   - AI still benefits from step-by-step reasoning
   - Users get clean, concise results

2. **Production Ready:**
   - Responses are polished and professional
   - No verbose explanations cluttering the output
   - Token-efficient for cost-sensitive applications

3. **Flexible Control:**
   - Keep `<thinking>` for debugging and logging
   - Show `<output>` to end users
   - Audit AI reasoning when needed

4. **User Experience:**
   - Faster to read and understand
   - More professional appearance
   - Reduces cognitive load on users

**Real-World Applications:**

- **Code Generation Tools:** IDE extensions that generate clean code without explanations
- **Automated PR Reviews:** Concise comments on pull requests with reasoning logged separately
- **Documentation Generators:** Clean docs without showing the analysis process
- **Customer Support Bots:** Helpful answers without exposing decision trees
- **API Code Examples:** Clean, copy-paste ready code snippets
- **Debugging Assistants:** Direct fixes without lengthy troubleshooting narratives

**Implementation Pattern:**

```python
system_prompt = """
Process:
1. In <thinking> tags: Analyze, plan, consider edge cases
2. In <output> tags: Provide only the final result

Never show <thinking> to users - it's for your internal process only.
"""

# Then extract: 
output = extract_tag(response, 'output')  # Show to user
thinking = extract_tag(response, 'thinking')  # Log for debugging
```

**Pro Tip:** You can combine inner monologue with other tactics! Use it with prompt chaining for multi-step workflows where each step produces clean output, or with role prompting for specialized expert responses without verbose explanations.


---

### 🎯 Try It Yourself: Inner Monologue

**Common Misconception:** Users need to see all of AI's reasoning and thought process.

**The Reality:** Clean outputs with hidden reasoning provide better UX while maintaining quality.

**Your Task:** Below, AI generates code with verbose explanations mixed in. Fix it by:
1. Instructing AI to use `<thinking>` tags for internal reasoning
2. Using `<output>` tags for the final clean code
3. Extracting only the `<output>` section for the user

Perfect for production tools where users want code, not essays!

In [ ]:
# ❌ BAD: Verbose output with explanations mixed in
bad_messages = [{
    "role": "user",
    "content": "Write a function to validate email addresses with regex."
}]
bad_response = get_chat_completion(bad_messages)
print("=" * 70)
print("VERBOSE OUTPUT (Explanations mixed with code):")
print("=" * 70)
print(bad_response)
print("\n")

# ✅ YOUR TURN: Use inner monologue to separate thinking from output
# TODO: Uncomment and complete
# good_messages = [{
#     "role": "system",
#     "content": """You are a code generator. Process:
# 1. In <thinking> tags: Plan the solution, consider edge cases
# 2. In <output> tags: Provide only the final, clean code

# Users see only <output>. Keep thinking internal."""
#     },
#     {
#         "role": "user",
#         "content": "Write a function to validate email addresses with regex."
#     }
# ]
# good_response = get_chat_completion(good_messages)

# # Extract clean output for user
# import re
# output_match = re.search(r'<output>(.*?)</output>', good_response, re.DOTALL)
# if output_match:
#     clean_output = output_match.group(1).strip()
#     print("=" * 70)
#     print("CLEAN OUTPUT (What user sees):")
#     print("=" * 70)
#     print(clean_output)
    
#     # Thinking is logged but not shown to user
#     thinking_match = re.search(r'<thinking>(.*?)</thinking>', good_response, re.DOTALL)
#     if thinking_match:
#         print("\n[Logged internally for debugging]:")
#         print(thinking_match.group(1).strip()[:200] + "...")

# print("\n💡 Users get clean code, you keep the reasoning for debugging!")

---

<div style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); color: white; padding: 24px; border-radius: 12px; margin: 40px 0; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <div style="text-align: center; margin-bottom: 20px;">
    <h2 style="color: white; margin: 0; font-size: 1.8em; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">🎯 Ready for Hands-On Practice?</h2>
    <p style="margin: 8px 0; font-size: 1.1em; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">All 8 tactics learned! • ~20-30 minutes for practice</p>
  </div>
  
  <div style="background: rgba(0,0,0,0.25); padding: 16px; border-radius: 8px; margin: 16px 0;">
    <p style="margin: 8px 0; font-size: 1.05em; font-weight: 600; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">🎉 Congratulations! You've Learned:</p>
    <ul style="margin: 8px 0; padding-left: 24px; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">
      <li>Tactic 0: Write Clear Instructions</li>
      <li>Tactic 1: Role Prompting</li>
      <li>Tactic 2: Structured Inputs</li>
      <li>Tactic 3: Few-Shot Examples</li>
      <li>Tactic 4: Chain-of-Thought</li>
      <li>Tactic 5: Reference Citations</li>
      <li>Tactic 6: Prompt Chaining</li>
      <li>Tactic 7: LLM-as-Judge</li>
      <li>Tactic 8: Inner Monologue</li>
    </ul>
  </div>
  
  <div style="background: rgba(0,0,0,0.25); padding: 16px; border-radius: 8px; margin: 16px 0;">
    <p style="margin: 8px 0; font-size: 1.05em; font-weight: 600; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">⏭️ What's Next:</p>
    <ul style="margin: 8px 0; padding-left: 24px; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">
      <li><strong>Activity 2.1:</strong> Role Prompting + Structured Inputs</li>
      <li><strong>Activity 2.2:</strong> Few-Shot Examples + Chain-of-Thought</li>
      <li><strong>Activity 2.3:</strong> Reference Citations + Prompt Chaining</li>
      <li><strong>Activity 2.4:</strong> LLM-as-Judge + Inner Monologue</li>
    </ul>
    <p style="margin: 12px 0 0 0; font-size: 0.95em; text-shadow: 1px 1px 2px rgba(0,0,0,0.2);">💪 Time to apply what you've learned!</p>
  </div>
  
  <div style="background: rgba(255,255,255,0.95); padding: 14px; border-radius: 8px; margin: 16px 0; text-align: center; color: #1e293b;">
    <p style="margin: 0; font-weight: bold; font-size: 1.1em; color: #1e293b;">📌 BOOKMARK TO RESUME:</p>
    <p style="margin: 8px 0 0 0; font-size: 1.15em; font-weight: bold; color: #0f172a;">"Hands-On Practice - Activity 2.1"</p>
  </div>
  
  <p style="text-align: center; margin: 16px 0 0 0; font-size: 0.9em; text-shadow: 1px 1px 2px rgba(0,0,0,0.3);">
    💡 <em>The practice activities reinforce learning. Take a break if needed before diving in!</em>
  </p>
</div>

---



---

## 🏃‍♀️ Hands-On Practice

Now let's practice what you've learned! These exercises will help you master the 8 core tactics.

### Activity 2.1: Role Prompting & Structured Inputs

**Goal:** Combine role prompting with XML delimiters to organize multi-file code analysis.

**Your Task:** Create a prompt that uses a QA Engineer persona to analyze test coverage for multiple files.

In [ ]:
# HINT: Combine Tactic 1 (Role Prompting) + Tactic 2 (Structured Inputs)
# - Use system message to define QA Engineer role
# - Use XML tags: <test_file>, <source_code>, <requirements>
# - Ask for structured output with coverage analysis

test_file = """
def calculate_total(items, tax_rate=0.1):
    subtotal = sum(item['price'] * item['quantity'] for item in items)
    return subtotal * (1 + tax_rate)
"""

source_code = """
class ShoppingCart:
    def __init__(self):
        self.items = []
    
    def add_item(self, name, price, quantity=1):
        self.items.append({'name': name, 'price': price, 'quantity': quantity})
    
    def get_total(self, tax_rate=0.1):
        return calculate_total(self.items, tax_rate)
"""

# YOUR TASK: Create messages using role prompting + XML structure
# messages = [
#     {"role": "system", "content": "..."}, 
#     {"role": "user", "content": "..."}
# ]
# response = get_chat_completion(messages)
# print(response)


### Activity 2.2: Few-Shot Examples & Chain-of-Thought

**Goal:** Use examples to teach AI your coding style, then apply chain-of-thought for analysis.

**Your Task:** Provide 3 examples of your preferred error message format, then ask AI to generate error messages for a new function using step-by-step reasoning.


In [ ]:
# HINT: Combine Tactic 3 (Few-Shot) + Tactic 4 (Chain-of-Thought)
# - Provide 3 examples of error messages in your preferred style
# - Use <example> tags for each
# - Add "Think step-by-step" instruction
# - Ask AI to analyze a new function and generate error messages

new_function = """
def transfer_funds(from_account, to_account, amount, currency='USD'):
    if amount <= 0:
        raise ValueError("Amount must be positive")
    if from_account == to_account:
        raise ValueError("Cannot transfer to same account")
    # Transfer logic here...
"""

# YOUR TASK: Create messages with few-shot examples + CoT reasoning
# Example format you want:
# - "ERROR [CODE]: Human-readable message. Suggestion: ..."
# 
# messages = [...]
# response = get_chat_completion(messages)
# print(response)


### Activity 2.3: Reference Citations & Prompt Chaining

**Goal:** Build a 2-step prompt chain that analyzes documentation and generates code.

**Your Task:** 
- **Step 1:** Extract relevant quotes from API docs about authentication
- **Step 2:** Use those quotes to generate secure authentication code


In [ ]:
# HINT: Combine Tactic 5 (Reference Citations) + Tactic 6 (Prompt Chaining)
# - STEP 1: Extract quotes about auth requirements
# - STEP 2: Use extracted quotes to generate implementation

api_documentation = """
# Authentication API v2

## Security Requirements
All API requests must include:
- API key in X-API-Key header
- Request signature using HMAC-SHA256
- Timestamp within 5 minutes of server time
- Rate limiting: 100 requests per minute per key

## Key Management
- Store keys in environment variables, never in code
- Rotate keys every 90 days
- Use separate keys for dev/staging/production

## Error Handling
- 401: Invalid or missing API key
- 403: Valid key but insufficient permissions
- 429: Rate limit exceeded
"""

# YOUR TASK: Create a 2-step chain
# STEP 1: Extract relevant quotes in <quotes> tags
# step1_messages = [...]
# quotes = get_chat_completion(step1_messages)
# print("STEP 1 - Extracted Quotes:")
# print(quotes)
# print("\n")

# STEP 2: Generate code using the quotes
# step2_messages = [...]  # Pass quotes from step 1
# code = get_chat_completion(step2_messages)
# print("STEP 2 - Generated Code:")
# print(code)


### Activity 2.4: LLM-as-Judge & Inner Monologue

**Goal:** Create a self-critique loop with clean final output.

**Your Task:** 
- Generate a function with potential issues
- Use AI as judge to critique it with weighted criteria
- Get improved version with inner monologue (only show final code)


In [ ]:
# HINT: Combine Tactic 7 (LLM-as-Judge) + Tactic 8 (Inner Monologue)
# Create a 3-step process:
# - STEP 1: Generate initial code
# - STEP 2: Judge it with weighted criteria (Security 40%, Performance 30%, Readability 30%)
# - STEP 3: Improve using <thinking> and <output> tags

requirement = "Create a function to validate and sanitize user email input"

# YOUR TASK: Build the 3-step self-improvement loop
# STEP 1: Generate initial implementation
# step1_messages = [...]
# initial_code = get_chat_completion(step1_messages)
# print("STEP 1 - Initial Code:")
# print(initial_code)
# print("\n")

# STEP 2: Critique with weighted rubric
# step2_messages = [...]  # Define criteria with weights
# critique = get_chat_completion(step2_messages)
# print("STEP 2 - Critique:")
# print(critique)
# print("\n")

# STEP 3: Improve with inner monologue
# step3_messages = [...]  # Use <thinking> and <output> tags
# improved = get_chat_completion(step3_messages)
# 
# # Extract only the <output> section
# import re
# output_match = re.search(r'<output>(.*?)</output>', improved, re.DOTALL)
# if output_match:
#     print("STEP 3 - Improved Code (Clean Output):")
#     print(output_match.group(1).strip())


### 🎯 Exercise Solutions & Discussion

<div style="margin-top:16px; color:#78350f; padding:12px; background:#fef3c7; border-radius:6px; border-left:4px solid #f59e0b;">
<strong>💡 Try the exercises above first!</strong> <br><br>
Complete Activities 2.1-2.4 before checking the solutions below.
</div>


<details>
<summary><strong>📋 Click to reveal solutions</strong></summary>

**Activity 2.1 Solution:**
```python
messages = [
    {
        "role": "system",
        "content": "You are a QA engineer analyzing test coverage. Provide detailed coverage recommendations."
    },
    {
        "role": "user",
        "content": f"""
<test_file>
{test_file}
</test_file>

<source_code>
{source_code}
</source_code>

<requirements>
- Test happy path scenarios
- Test edge cases (empty lists, zero quantities)
- Test error conditions
- Verify tax calculations
</requirements>

Analyze test coverage and identify missing test cases. Format response as:
1. Current Coverage Assessment
2. Missing Test Scenarios
3. Recommended Test Cases
"""
    }
]
```

**Activity 2.2 Solution:**
```python
messages = [
    {"role": "system", "content": "Generate error messages following the provided examples."},
    {"role": "user", "content": "Generate error message for invalid email"},
    {"role": "assistant", "content": "ERROR [E001]: Invalid email format. Suggestion: Use format 'user@domain.com'"},
    
    {"role": "user", "content": "Generate error message for empty field"},
    {"role": "assistant", "content": "ERROR [E002]: Required field is empty. Suggestion: Provide a valid value"},
    
    {"role": "user", "content": "Generate error message for duplicate entry"},
    {"role": "assistant", "content": "ERROR [E003]: Duplicate entry detected. Suggestion: Use a unique identifier"},
    
    {"role": "user", "content": f"Analyze this function step-by-step and generate appropriate error messages:\n{new_function}"}
]
```

**Activity 2.3 Solution:**
```python
# STEP 1
step1_messages = [{
    "role": "user",
    "content": f"""<documentation>{api_documentation}</documentation>

Extract key quotes about authentication requirements in <quotes> tags."""
}]

# STEP 2 (pass quotes from step 1)
step2_messages = [{
    "role": "user",
    "content": f"""Based on these requirements:
{quotes}

Generate Python code implementing secure authentication. Use <thinking> for analysis and <output> for code."""
}]
```

**Activity 2.4 Solution:**
```python
# STEP 2 - Judge
step2_messages = [{
    "role": "system",
    "content": """You are a code quality judge. Evaluate based on:
- Security (40%)
- Performance (30%)  
- Readability (30%)

Provide scores and specific issues."""
}]

# STEP 3 - Improve with inner monologue
step3_messages = [{
    "role": "user",
    "content": f"""Improve this code addressing the critique:

{initial_code}

Critique: {critique}

Process:
1. In <thinking> tags: Analyze issues and plan improvements
2. In <output> tags: Provide only the final improved code"""
}]
```

**Key Takeaways:**
- Tactics work better combined than alone
- XML tags organize complex information
- Chains enable multi-step reasoning
- Inner monologue keeps output clean

</details>


🎉 **Excellent work!** You've practiced combining multiple tactics to solve real-world coding challenges.

**What you've demonstrated:**
- ✅ Combined role prompting with structured inputs (Activity 2.1)
- ✅ Used few-shot examples with chain-of-thought (Activity 2.2)
- ✅ Built prompt chains with reference citations (Activity 2.3)
- ✅ Created self-improvement loops with clean output (Activity 2.4)

---

## 📈 Track Your Progress

> **💡 New to Skills Checklists?** See [Tracking Your Progress](../../README.md#-tracking-your-progress) in the main README for details on how the Skills Checklist works and when to check off skills.

### Self-Assessment Questions

After completing Module 2, ask yourself:
1. Can I explain how role prompting improves AI responses?
2. Can I use delimiters (XML tags) effectively to organize complex inputs?
3. Can I create few-shot examples to establish consistent styles?
4. Can I implement chain-of-thought reasoning for systematic analysis?
5. Can I ground AI responses in reference texts with proper citations?
6. Can I break complex tasks into sequential prompt chains?
7. Can I use LLM-as-Judge to evaluate and improve code quality?
8. Can I implement inner monologue to separate reasoning from final output?

### Progress Overview

<div style="margin-bottom:12px; padding:10px; background:#fef3c7; border-left:4px solid #f59e0b; border-radius:4px; color:#78350f;">
<strong>💡 Note:</strong> The status indicators below (✅/⬜) are <strong>visual guides only</strong> and cannot be clicked. Scroll down to "<strong>Check Off Your Skills</strong>" for the interactive checkboxes where you'll track your actual progress!
</div>

<div style="background:#f8f9fa; border-radius:8px; padding:18px 22px; margin-bottom:18px; border:1px solid #e0e0e0; box-shadow:0 1px 4px #0001; color:#000000">

<span style="color:#000000; font-weight:600">**Module 2 Skills Checklist:**</span> 
<div style="margin:10px 0; font-size:0.9em; color:#666">Track your progress by checking off skills below. When you master all 16 skills (2 per tactic), you'll have achieved 100% completion!</div>

**Current Status:**
- <span style="color:#059669">✅ Environment Setup (Tutorial Completed)</span>
- <span style="color:#059669">✅ 8 Core Techniques Learned (Tutorial Completed)</span>  
- <span style="color:#e97316">⬜ Skills Mastery (Use Skills Checklist below)</span>

**Progress Guide:**
- <span style="color:#666; font-size:0.9em">0-4 skills checked: Beginner (25-38%)</span>
- <span style="color:#666; font-size:0.9em">5-8 skills checked: Developing (44-56%)</span>
- <span style="color:#666; font-size:0.9em">9-12 skills checked: Intermediate (63-75%)</span>
- <span style="color:#666; font-size:0.9em">13-15 skills checked: Advanced (81-94%)</span>
- <span style="color:#666; font-size:0.9em">16 skills checked: Expert (100%) 🎉</span>

<span style="color:#000000; font-weight:600">**Module 3:**</span> <span style="color:#888888; font-weight:600">Coming Next</span>
- <span style="color:#888888">⬜ Advanced Applications</span>
- <span style="color:#888888">⬜ Complex Refactoring Scenarios</span>
- <span style="color:#888888">⬜ Testing and QA Workflows</span>
- <span style="color:#888888">⬜ Production Debugging Prompts</span>

</div>

### Check Off Your Skills

<div style="background:#f8f9fa; border-radius:8px; padding:18px 22px; margin-bottom:18px; border:1px solid #e0e0e0; box-shadow:0 1px 4px #0001; color:#000000">

<span style="color:#000000">Mark each skill as you master it (2 skills per tactic = 16 total):</span>

<span style="color:#000000; font-weight:600">**1. Role Prompting:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill1" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can create effective software engineering personas (security, performance, QA)</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill2" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can assign specific expertise roles to get specialized analysis</span>
</div>

<span style="color:#000000; font-weight:600">**2. Structured Inputs:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill3" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can use delimiters (### or XML) to organize complex inputs</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill4" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can handle multi-file scenarios with clear structure</span>
</div>

<span style="color:#000000; font-weight:600">**3. Few-Shot Examples:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill5" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can create few-shot examples to establish consistent response styles</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill6" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can use examples to teach AI my coding standards and documentation formats</span>
</div>

<span style="color:#000000; font-weight:600">**4. Chain-of-Thought:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill7" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can implement step-by-step reasoning for systematic analysis</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill8" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can force AI to work through problems before judging solutions</span>
</div>

<span style="color:#000000; font-weight:600">**5. Reference Citations:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill9" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can structure multi-document prompts with proper XML tags</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill10" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can request quote extraction before analysis to reduce hallucinations</span>
</div>

<span style="color:#000000; font-weight:600">**6. Prompt Chaining:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill11" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can break complex tasks into sequential prompt chains</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill12" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can pass context between chain steps using structured tags</span>
</div>

<span style="color:#000000; font-weight:600">**7. LLM-as-Judge:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill13" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can create evaluation rubrics with weighted criteria for code assessment</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill14" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can implement self-critique loops for iterative improvement</span>
</div>

<span style="color:#000000; font-weight:600">**8. Inner Monologue:**</span>
<div style="margin:8px 0">
- <input type="checkbox" id="skill15" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can separate thinking from output using &lt;thinking&gt; and &lt;output&gt; tags</span>
</div>
<div style="margin:8px 0">
- <input type="checkbox" id="skill16" onchange="this.nextSibling.style.textDecoration = this.checked ? 'line-through' : 'none'; this.nextSibling.style.color = this.checked ? '#888888' : '#000000'; this.nextSibling.style.fontWeight = this.checked ? 'normal' : '500';"> <span style="color:#000000; font-weight:500">I can extract clean outputs while logging reasoning for debugging</span>
</div>

</div>

<div style="margin-top:16px; padding:12px; background:#dbeafe; border-radius:6px; border-left:4px solid #3b82f6; color:#000000">
<strong>💡 Remember:</strong> The goal is not just to complete activities, but to build lasting skills that transform your development workflow!
</div>


🎉 **Outstanding!** You've completed Module 2 and learned **eight powerful prompt engineering tactics**:

1. 🎭 **Role Prompting** - Transform AI into specialized domain experts
2. 📋 **Structured Inputs** - Organize complex scenarios with XML delimiters
3. 📚 **Few-Shot Examples** - Teach AI your preferred styles and standards
4. ⛓️‍💥 **Chain-of-Thought** - Guide systematic step-by-step reasoning
5. 📖 **Reference Citations** - Ground responses in actual documentation to reduce hallucinations
6. 🔗 **Prompt Chaining** - Break complex tasks into sequential workflows
7. ⚖️ **LLM-as-Judge** - Create evaluation rubrics and self-critique loops
8. 🤫 **Inner Monologue** - Separate reasoning from clean final outputs

This is exactly how professional developers use AI assistants for real-world software engineering tasks!

## 🎊 Module 2 Complete!

### What You've Accomplished

- ✅ **Practiced about role prompting** and saw how personas provide specialized expertise
- ✅ **Used structured delimiters** to organize complex, multi-part inputs with XML tags
- ✅ **Applied few-shot examples** to establish consistent response styles
- ✅ **Implemented chain-of-thought reasoning** for systematic analysis
- ✅ **Grounded AI responses** in reference texts with proper citations
- ✅ **Built prompt chains** to break complex tasks into sequential steps
- ✅ **Used LLM-as-Judge** for evaluating and improving code quality
- ✅ **Implemented inner monologue** to separate reasoning from final output

### Next Steps

Continue to **Module 3: Advanced Software Engineering Applications** where you'll learn:
- Implement prompts for code review, debugging, documentation, and refactoring
- Design reusable prompt templates for software engineering workflows
- Evaluate prompt effectiveness and output quality
- Refine templates based on feedback and edge cases
- Apply best practices for SDLC integration
